<a href="https://colab.research.google.com/github/naga-dheeraj-p/Assignments/blob/main/Final_BackTesting%2B%20oi_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [132]:
data_=pd.read_csv('/content/gdrive/MyDrive/Final_Merged_Data/all_data_combined.csv')
data_

,datetime,atm,open,high,low,close,volume,oi,max_pain_strike,intraday_pcr,...,highest_above_vah,lowest_below_val,gap_type,gap_filled,3_day_vah,5_day_vah,3_day_val,5_day_val,3_day_poc,5_day_poc
0,2018-01-01 09:15:00,25570.00,25634.00,25643.10,25608.00,25643.10,480.0,0.0,25700,0.8287,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 09:20:00,25570.00,25631.50,25675.70,25631.50,25665.30,600.0,0.0,25600,0.7786,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01 09:25:00,25570.00,25665.30,25665.30,25655.10,25655.10,240.0,0.0,25700,0.7420,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01 09:30:00,25570.00,25647.70,25647.70,25647.70,25647.70,240.0,0.0,25600,0.7683,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01 09:35:00,25570.00,25647.70,25647.70,25647.00,25647.00,440.0,0.0,25700,0.7973,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92694,2022-12-30 15:10:00,43501.05,43320.05,43349.70,43302.45,43322.00,2650.0,168225.0,42500,0.6826,...,43731.8,43165.0,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7
92695,2022-12-30 15:15:00,43501.05,43317.30,43331.95,43290.00,43310.00,4950.0,168625.0,42000,0.6204,...,43731.8,43165.0,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7
92696,2022-12-30 15:20:00,43501.05,43307.35,43443.95,43293.25,43432.90,6600.0,168750.0,42500,0.5152,...,43731.8,43165.0,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7
92697,2022-12-30 15:25:00,43501.05,43433.60,43443.70,43377.05,43403.40,6100.0,168500.0,42000,0.4360,...,43731.8,43165.0,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7


In [16]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'data_' is already defined and available
data = data_
data.sort_values(by='datetime', inplace=True)
data['datetime'] = pd.to_datetime(data['datetime'])

# Group by date
data['date'] = data['datetime'].dt.date
daily_data = data.groupby('date').agg({
    '3_day_vah': 'last',
    '3_day_val': 'last'
}).reset_index()

# Shift the 3-day VAH and VAL values
daily_data['prev_3_day_vah'] = daily_data['3_day_vah'].shift(1)
daily_data['prev_3_day_val'] = daily_data['3_day_val'].shift(1)
daily_data.dropna(subset=['prev_3_day_vah', 'prev_3_day_val'], inplace=True)

# Merge the shifted VAH and VAL back to the original data
data = data.merge(daily_data[['date', 'prev_3_day_vah', 'prev_3_day_val']], on='date', how='left')
data.dropna(subset=['prev_3_day_vah', 'prev_3_day_val'], inplace=True)

trades = []

def calculate_trades_with_fixed_exit(df, initial_investment):
    entry_price = 0
    trade_type = None
    investment_value = initial_investment
    investment_values = []
    investment_times = []

    current_trade_day = None

    for index, row in df.iterrows():
        trade_day = row['date']

        # Check if a trade is already executed for the day
        if trade_day == current_trade_day:
            continue

        # Check for short selling entry
        if row['close'] > 1.01 * row['prev_3_day_vah'] and trade_type is None:
            entry_price = row['close']
            trade_type = 'short'
            entry_time = row['datetime']
            year = entry_time.year
            month = entry_time.month
            current_trade_day = trade_day
        # Check for long position entry
        elif row['close'] < 0.99 * row['prev_3_day_val'] and trade_type is None:
            entry_price = row['close']
            trade_type = 'long'
            entry_time = row['datetime']
            year = entry_time.year
            month = entry_time.month
            current_trade_day = trade_day

        # Execute trade if a position was taken
        if trade_type is not None:
            end_of_day = row['datetime'].replace(hour=15, minute=15, second=0)
            future_df = df[(df['datetime'] > row['datetime']) & (df['datetime'] <= end_of_day)]
            exit_row = None

            # Check for the additional exit condition
            for future_index, future_row in future_df.iterrows():
                close_price = future_row['close']
                prev_vah_zone_upper = future_row['prev_3_day_vah'] * 1.001
                prev_vah_zone_lower = future_row['prev_3_day_vah'] * 0.999
                prev_val_zone_upper = future_row['prev_3_day_val'] * 1.001
                prev_val_zone_lower = future_row['prev_3_day_val'] * 0.999

                if trade_type == 'short' and prev_vah_zone_lower <= close_price <= prev_vah_zone_upper:
                    exit_row = future_row
                    break
                elif trade_type == 'long' and prev_val_zone_lower <= close_price <= prev_val_zone_upper:
                    exit_row = future_row
                    break

            # If no exit condition is met, exit at the end of the day
            if exit_row is None and not future_df.empty:
                exit_row = future_df.iloc[-1]

            if exit_row is not None:
                exit_price = exit_row['close']
                profit_loss = exit_price - entry_price if trade_type == 'long' else entry_price - exit_price
                profit_loss_percentage = (profit_loss / entry_price) * 100
                investment_value += profit_loss
                trades.append({
                    'entry_price': entry_price,
                    'exit_price': exit_price,
                    'profit_loss': profit_loss,
                    'trade_type': trade_type,
                    'entry_time': entry_time,
                    'exit_time': exit_row['datetime'],
                    'year': year,
                    'month': month,
                    'profit_loss_percentage': profit_loss_percentage
                })
                investment_values.append(investment_value)
                investment_times.append(exit_row['datetime'])
                entry_price = 0
                trade_type = None

    return investment_values, investment_times

investment_values, investment_times = calculate_trades_with_fixed_exit(data, initial_investment=100000)

trades_df = pd.DataFrame(trades)

yearly_profit_loss_percentage = trades_df.groupby('year')['profit_loss_percentage'].sum()
total_profit_loss_percentage = trades_df['profit_loss_percentage'].sum()

# Print the results
print("Yearly Profit/Loss Percentage:")
print(yearly_profit_loss_percentage)
print("\nTotal Profit/Loss Percentage:")
print(total_profit_loss_percentage)

# Display the trades DataFrame
print("\nTrades DataFrame:")
trades_df


Yearly Profit/Loss Percentage:
year
2018     3.030374
2019     2.901618
2020   -26.437061
2021    -4.906007
2022    17.931963
Name: profit_loss_percentage, dtype: float64

Total Profit/Loss Percentage:
-7.479112182223526

Trades DataFrame:


,entry_price,exit_price,profit_loss,trade_type,entry_time,exit_time,year,month,profit_loss_percentage
0,26051.90,26085.40,-33.50,short,2018-01-15 11:20:00,2018-01-15 15:15:00,2018,1,-0.128589
1,26315.00,26337.90,-22.90,short,2018-01-17 14:45:00,2018-01-17 15:15:00,2018,1,-0.087023
2,26685.00,26559.50,125.50,short,2018-01-18 09:15:00,2018-01-18 15:15:00,2018,1,0.470302
3,26690.30,26934.50,-244.20,short,2018-01-19 10:10:00,2018-01-19 15:15:00,2018,1,-0.914939
4,26999.20,27066.90,-67.70,short,2018-01-22 10:00:00,2018-01-22 15:15:00,2018,1,-0.250748
...,...,...,...,...,...,...,...,...,...
633,43502.80,43726.10,223.30,long,2022-12-19 09:50:00,2022-12-19 15:15:00,2022,12,0.513300
634,42899.00,42973.25,74.25,long,2022-12-21 13:55:00,2022-12-21 15:15:00,2022,12,0.173081
635,42886.90,42694.55,-192.35,long,2022-12-22 09:30:00,2022-12-22 15:15:00,2022,12,-0.448505
636,42377.90,41940.85,-437.05,long,2022-12-23 09:15:00,2022-12-23 15:15:00,2022,12,-1.031316


# Rolling Reversion Strategy

In [20]:
import pandas as pd

# Assuming 'data_' is already defined and available
data = data_
data.sort_values(by='datetime', inplace=True)
data['datetime'] = pd.to_datetime(data['datetime'])

data['prev_3_day_vah'] = data['3_day_vah'].shift(1)
data['prev_3_day_val'] = data['3_day_val'].shift(1)
data.dropna(subset=['prev_3_day_vah', 'prev_3_day_val'], inplace=True)

trades = []

def calculate_trades_with_fixed_exit(df):
    entry_price = 0
    trade_type = None
    exit_reason = None
    vah_or_val_crossed = None
    vah_or_val_value = None

    for index, row in df.iterrows():
        # Check for short selling entry
        if row['close'] > 1.01 * row['prev_3_day_vah'] and trade_type is None:
            entry_price = row['close']
            trade_type = 'short'
            entry_time = row['datetime']
            year = entry_time.year
            month = entry_time.month
            vah_or_val_crossed = 'prev_3_day_vah'
            vah_or_val_value = row['prev_3_day_vah']

        # Check for long position entry
        elif row['close'] < 0.99 * row['prev_3_day_val'] and trade_type is None:
            entry_price = row['close']
            trade_type = 'long'
            entry_time = row['datetime']
            year = entry_time.year
            month = entry_time.month
            vah_or_val_crossed = 'prev_3_day_val'
            vah_or_val_value = row['prev_3_day_val']

        # Check for exit condition
        if trade_type is not None:
            if (trade_type == 'short' and row['close'] <= 1.001 * row['prev_3_day_vah']) or (trade_type == 'long' and row['close'] >= 0.999 * row['prev_3_day_val']):
                exit_price = row['close']
                profit_loss = exit_price - entry_price if trade_type == 'long' else entry_price - exit_price
                profit_loss_percentage = (profit_loss / entry_price) * 100
                exit_reason = 'target hit'
                trades.append({
                    'entry_price': entry_price,
                    'exit_price': exit_price,
                    'profit_loss': profit_loss,
                    'trade_type': trade_type,
                    'entry_time': entry_time,
                    'exit_time': row['datetime'],
                    'year': year,
                    'month': month,
                    'profit_loss_percentage': profit_loss_percentage,
                    'vah_or_val_crossed': vah_or_val_crossed,
                    'vah_or_val_value': vah_or_val_value,
                    'exit_reason': exit_reason
                })
                entry_price = 0
                trade_type = None
                exit_reason = None
                vah_or_val_crossed = None
                vah_or_val_value = None

            # Exit at 15:15:00 if trade is still open
            if row['datetime'].time() >= pd.Timestamp('15:15:00').time() and trade_type is not None:
                exit_price = row['close']
                profit_loss = exit_price - entry_price if trade_type == 'long' else entry_price - exit_price
                profit_loss_percentage = (profit_loss / entry_price) * 100
                exit_reason = 'end of day'
                trades.append({
                    'entry_price': entry_price,
                    'exit_price': exit_price,
                    'profit_loss': profit_loss,
                    'trade_type': trade_type,
                    'entry_time': entry_time,
                    'exit_time': row['datetime'],
                    'year': year,
                    'month': month,
                    'profit_loss_percentage': profit_loss_percentage,
                    'vah_or_val_crossed': vah_or_val_crossed,
                    'vah_or_val_value': vah_or_val_value,
                    'exit_reason': exit_reason
                })
                entry_price = 0
                trade_type = None
                exit_reason = None
                vah_or_val_crossed = None
                vah_or_val_value = None

calculate_trades_with_fixed_exit(data)
trades_df = pd.DataFrame(trades)

# Remove rows with 0 profit_loss_percentage
trades_df = trades_df[trades_df['profit_loss_percentage'] != 0]

yearly_profit_loss = trades_df.groupby('year')['profit_loss'].sum()
yearly_profit_loss_percentage = trades_df.groupby('year')['profit_loss_percentage'].sum()

monthly_profit_loss = trades_df.groupby(['year', 'month'])['profit_loss'].sum()
monthly_profit_loss_percentage = trades_df.groupby(['year', 'month'])['profit_loss_percentage'].sum()

total_profit_loss = trades_df['profit_loss'].sum()
total_profit_loss_percentage = trades_df['profit_loss_percentage'].sum()

print("\nYearly Profit/Loss:")
print(yearly_profit_loss)
print("\nYearly Profit/Loss Percentage:")
print(yearly_profit_loss_percentage)
print("\nMonthly Profit/Loss:")
print(monthly_profit_loss)
print("\nMonthly Profit/Loss Percentage:")
print(monthly_profit_loss_percentage)
print("\nTotal Profit/Loss:")
print(total_profit_loss)
print("\nTotal Profit/Loss Percentage:")
print(total_profit_loss_percentage)
print("\nTrades DataFrame:")
trades_df.head(60)



Yearly Profit/Loss:
year
2018    2849.40
2019    1554.70
2020    3890.00
2021    5454.85
2022    5725.25
Name: profit_loss, dtype: float64

Yearly Profit/Loss Percentage:
year
2018    11.345443
2019     5.336475
2020    17.759595
2021    15.711757
2022    15.720520
Name: profit_loss_percentage, dtype: float64

Monthly Profit/Loss:
year  month
2018  1         125.50
      2         799.00
      3         661.10
      4         -57.70
      5         399.90
      6         393.80
      7         -63.40
      8          37.60
      9        -340.10
      10        553.90
      12        339.80
2019  1        -180.15
      2         117.00
      3         740.15
      4        -253.00
      5          90.40
      6        -134.75
      7        -298.60
      8         759.40
      9        -803.35
      10        937.75
      11        299.35
      12        280.50
2020  1         509.60
      2        -342.10
      3        -455.20
      4         668.70
      5       -1207.35
      6   

,entry_price,exit_price,profit_loss,trade_type,entry_time,exit_time,year,month,profit_loss_percentage,vah_or_val_crossed,vah_or_val_value,exit_reason
0,26685.00,26559.50,125.50,short,2018-01-18 09:15:00,2018-01-18 15:15:00,2018,1,0.470302,prev_3_day_vah,26174.78,end of day
4,26965.00,26517.20,-447.80,long,2018-02-02 09:15:00,2018-02-02 15:15:00,2018,2,-1.660671,prev_3_day_val,27378.35,end of day
8,26130.00,26263.70,133.70,long,2018-02-05 09:15:00,2018-02-05 15:15:00,2018,2,0.511672,prev_3_day_val,27161.03,end of day
11,25360.90,26159.80,798.90,long,2018-02-06 09:15:00,2018-02-06 14:20:00,2018,2,3.150125,prev_3_day_val,26780.30,target hit
12,25449.50,25589.30,139.80,long,2018-02-09 09:55:00,2018-02-09 15:15:00,2018,2,0.549323,prev_3_day_val,25752.57,end of day
14,25200.10,25312.40,112.30,long,2018-02-19 09:15:00,2018-02-19 09:20:00,2018,2,0.445633,prev_3_day_val,25464.47,target hit
15,25025.40,25169.90,144.50,long,2018-02-19 10:00:00,2018-02-19 15:15:00,2018,2,0.577413,prev_3_day_val,25280.15,end of day
16,25426.00,25456.90,-30.90,short,2018-02-23 14:45:00,2018-02-23 15:15:00,2018,2,-0.121529,prev_3_day_vah,25162.22,end of day
19,25580.00,25810.00,-230.00,short,2018-02-26 09:15:00,2018-02-26 15:15:00,2018,2,-0.899140,prev_3_day_vah,25162.22,end of day
23,25786.80,25707.70,79.10,short,2018-02-27 09:15:00,2018-02-27 09:30:00,2018,2,0.306746,prev_3_day_vah,25431.18,target hit


In [33]:
import pandas as pd

# Assuming 'data_' is already defined and available
data = data_
data.sort_values(by='datetime', inplace=True)
data['datetime'] = pd.to_datetime(data['datetime'])

data['prev_3_day_vah'] = data['3_day_vah'].shift(1)
data['prev_3_day_val'] = data['3_day_val'].shift(1)
data.dropna(subset=['prev_3_day_vah', 'prev_3_day_val'], inplace=True)

trades = []

def calculate_trades_with_fixed_exit(df):
    entry_price = 0
    trade_type = None
    exit_reason = None
    vah_or_val_crossed = None
    vah_or_val_value = None
    stop_loss = None

    for index, row in df.iterrows():
        # Check for short selling entry
        if row['close'] > 1.01 * row['prev_3_day_vah'] and trade_type is None:
            entry_price = row['close']
            trade_type = 'short'
            entry_time = row['datetime']
            year = entry_time.year
            month = entry_time.month
            vah_or_val_crossed = 'prev_3_day_vah'
            vah_or_val_value = row['prev_3_day_vah']
            stop_loss = entry_price * 1.01  # 1% above entry price

        # Check for long position entry
        elif row['close'] < 0.99 * row['prev_3_day_val'] and trade_type is None:
            entry_price = row['close']
            trade_type = 'long'
            entry_time = row['datetime']
            year = entry_time.year
            month = entry_time.month
            vah_or_val_crossed = 'prev_3_day_val'
            vah_or_val_value = row['prev_3_day_val']
            stop_loss = entry_price * 0.99  # 1% below entry price

        # Check for stop loss hits and adjust trailing stop loss
        if trade_type is not None:
            if trade_type == 'long':
                stop_loss = max(stop_loss, row['close'] * 0.99)
                if row['low'] <= stop_loss:
                    exit_price = stop_loss
                    exit_reason = 'stop loss hit'
                elif row['close'] >= 1.1 * row['prev_3_day_val']:
                    exit_price = row['close']
                    exit_reason = 'target hit'
            elif trade_type == 'short':
                stop_loss = min(stop_loss, row['close'] * 1.01)
                if row['high'] >= stop_loss:
                    exit_price = stop_loss
                    exit_reason = 'stop loss hit'
                elif row['close'] <= 0.99 * row['prev_3_day_vah']:
                    exit_price = row['close']
                    exit_reason = 'target hit'

            if exit_reason is not None:
                profit_loss = exit_price - entry_price if trade_type == 'long' else entry_price - exit_price
                profit_loss_percentage = (profit_loss / entry_price) * 100
                trades.append({
                    'entry_price': entry_price,
                    'exit_price': exit_price,
                    'profit_loss': profit_loss,
                    'trade_type': trade_type,
                    'entry_time': entry_time,
                    'exit_time': row['datetime'],
                    'year': year,
                    'month': month,
                    'profit_loss_percentage': profit_loss_percentage,
                    'vah_or_val_crossed': vah_or_val_crossed,
                    'vah_or_val_value': vah_or_val_value,
                    'exit_reason': exit_reason
                })
                entry_price = 0
                trade_type = None
                exit_reason = None
                vah_or_val_crossed = None
                vah_or_val_value = None
                stop_loss = None

            # Exit at 15:15:00 if trade is still open
            if row['datetime'].time() >= pd.Timestamp('15:15:00').time() and trade_type is not None:
                exit_price = row['close']
                exit_reason = 'end of day'
                profit_loss = exit_price - entry_price if trade_type == 'long' else entry_price - exit_price
                profit_loss_percentage = (profit_loss / entry_price) * 100
                trades.append({
                    'entry_price': entry_price,
                    'exit_price': exit_price,
                    'profit_loss': profit_loss,
                    'trade_type': trade_type,
                    'entry_time': entry_time,
                    'exit_time': row['datetime'],
                    'year': year,
                    'month': month,
                    'profit_loss_percentage': profit_loss_percentage,
                    'vah_or_val_crossed': vah_or_val_crossed,
                    'vah_or_val_value': vah_or_val_value,
                    'exit_reason': exit_reason
                })
                entry_price = 0
                trade_type = None
                exit_reason = None
                vah_or_val_crossed = None
                vah_or_val_value = None
                stop_loss = None

calculate_trades_with_fixed_exit(data)
trades_df = pd.DataFrame(trades)

# Remove rows with 0 profit_loss_percentage
trades_df = trades_df[trades_df['profit_loss_percentage'] != 0]

yearly_profit_loss = trades_df.groupby('year')['profit_loss'].sum()
yearly_profit_loss_percentage = trades_df.groupby('year')['profit_loss_percentage'].sum()

monthly_profit_loss = trades_df.groupby(['year', 'month'])['profit_loss'].sum()
monthly_profit_loss_percentage = trades_df.groupby(['year', 'month'])['profit_loss_percentage'].sum()

total_profit_loss = trades_df['profit_loss'].sum()
total_profit_loss_percentage = trades_df['profit_loss_percentage'].sum()

print("\nYearly Profit/Loss:")
print(yearly_profit_loss)
print("\nYearly Profit/Loss Percentage:")
print(yearly_profit_loss_percentage)
'''print("\nMonthly Profit/Loss Percentage:")
print(monthly_profit_loss_percentage)'''
print("\nTotal Profit/Loss:")
print(total_profit_loss)
print("\nTotal Profit/Loss Percentage:")
print(total_profit_loss_percentage)
print("\nTrades DataFrame:")
trades_df.head(60)



Yearly Profit/Loss:
year
2018     2489.5065
2019    -1060.3480
2020   -11323.8355
2021     -230.4285
2022     5396.7965
Name: profit_loss, dtype: float64

Yearly Profit/Loss Percentage:
year
2018    10.078122
2019    -3.779773
2020   -59.685865
2021    -0.318721
2022    14.692507
Name: profit_loss_percentage, dtype: float64

Total Profit/Loss:
-4728.309000000081

Total Profit/Loss Percentage:
-39.01373091463209

Trades DataFrame:


,entry_price,exit_price,profit_loss,trade_type,entry_time,exit_time,year,month,profit_loss_percentage,vah_or_val_crossed,vah_or_val_value,exit_reason
0,26685.00,26559.5000,125.5000,short,2018-01-18 09:15:00,2018-01-18 15:15:00,2018,1,0.470302,prev_3_day_vah,26174.78,end of day
4,26965.00,26695.3500,-269.6500,long,2018-02-02 09:15:00,2018-02-02 14:35:00,2018,2,-1.000000,prev_3_day_val,27378.35,stop loss hit
5,26680.90,26517.2000,-163.7000,long,2018-02-02 14:40:00,2018-02-02 15:15:00,2018,2,-0.613548,prev_3_day_val,27161.03,end of day
9,26130.00,26263.7000,133.7000,long,2018-02-05 09:15:00,2018-02-05 15:15:00,2018,2,0.511672,prev_3_day_val,27161.03,end of day
12,25360.90,25184.4120,-176.4880,long,2018-02-06 09:15:00,2018-02-06 10:10:00,2018,2,-0.695906,prev_3_day_val,26780.30,stop loss hit
13,25150.30,25898.2020,747.9020,long,2018-02-06 10:15:00,2018-02-06 14:20:00,2018,2,2.973730,prev_3_day_val,26111.53,stop loss hit
14,25449.50,25589.3000,139.8000,long,2018-02-09 09:55:00,2018-02-09 15:15:00,2018,2,0.549323,prev_3_day_val,25752.57,end of day
16,25200.10,25059.2760,-140.8240,long,2018-02-19 09:15:00,2018-02-19 09:55:00,2018,2,-0.558823,prev_3_day_val,25464.47,stop loss hit
17,25025.40,25169.9000,144.5000,long,2018-02-19 10:00:00,2018-02-19 15:15:00,2018,2,0.577413,prev_3_day_val,25280.15,end of day
18,25426.00,25456.9000,-30.9000,short,2018-02-23 14:45:00,2018-02-23 15:15:00,2018,2,-0.121529,prev_3_day_vah,25162.22,end of day


In [32]:
trades_df.value_counts(['exit_reason'])

exit_reason  
end of day       444
stop loss hit    333
target hit        19
Name: count, dtype: int64

# POC Strategy(Final Checking Required and Find Max Drawdown and Sharpe Ratio)

In [35]:
import pandas as pd

# Assuming `data_` is your initial DataFrame
data = data_
data['datetime'] = pd.to_datetime(data['datetime'])
data['date'] = data['datetime'].dt.date
data['year'] = data['datetime'].dt.year  # Extract year for grouping
data.sort_values(by='datetime', inplace=True)
daily_data = data.groupby('date').agg({
    'poc': 'first',
}).reset_index()

daily_data['poc_change'] = daily_data['poc'].pct_change()

# Identify days where the POC is trending higher or lower according to new conditions
daily_data['poc_trending_higher'] = (
    (daily_data['poc'].shift(1) > daily_data['poc'].shift(2) * 1.002)
)
daily_data['poc_trending_lower'] = (
    (daily_data['poc'].shift(1) < daily_data['poc'].shift(2) * 0.998)
)

# Get dates when the conditions are met
trending_higher_dates = daily_data[daily_data['poc_trending_higher']]['date']
trending_lower_dates = daily_data[daily_data['poc_trending_lower']]['date']

results = []
near_ranges = [0.0125]

def get_previous_trading_day(date, all_dates):
    previous_dates = all_dates[all_dates < date]
    if not previous_dates.empty:
        return previous_dates.max()
    return None

unique_dates = pd.Series(data['date'].unique()).sort_values()

def process_day_with_stoploss(date, prev_date, strategy, local_extreme_fn, near_ranges):
    prev_day_data = data[data['date'] == prev_date]
    local_extreme = local_extreme_fn(prev_day_data['close'])
    daily_subset = data[data['date'] == date]

    for near_range in near_ranges:
        within_range = daily_subset[
            (daily_subset['open'] <= local_extreme * (1 + near_range)) &
            (daily_subset['open'] >= local_extreme * (1 - near_range))
        ]

        if not within_range.empty:
            entry_close = within_range['close'].iloc[-1]  # Last 5-min candle close within the range

            # Calculate stop-loss level
            if strategy == 'Higher High POC':
                stop_loss = entry_close * 0.985 # 1.5% below entry for long position
            else:
                stop_loss = entry_close * 1.015 # 1.5% above entry for short position

            # Check if stop-loss is breached during the day
            subsequent_data = daily_subset[daily_subset['datetime'] > within_range['datetime'].iloc[-1]]
            stop_loss_hit = False
            if strategy == 'Higher High POC':
                stop_loss_hit = subsequent_data['low'].min() <= stop_loss
            else:
                stop_loss_hit = subsequent_data['high'].max() >= stop_loss

            if stop_loss_hit:
                movement_pct = (((stop_loss - entry_close) / entry_close) * 100)  # Negative impact of stop-loss hit
                last_5min_close = stop_loss  # Use stop-loss as the last 5-min close in case of stop-loss hit           ####### CHECK
            else:
                # Exit at 15:15:00
                exit_time = pd.Timestamp(f'{date} 15:15:00')
                last_candle_before_exit = daily_subset[daily_subset['datetime'] <= exit_time].iloc[-1]
                last_5min_close = last_candle_before_exit['close']
                movement_pct = ((last_5min_close - entry_close) / entry_close) * 100 if strategy == 'Higher High POC' else -((last_5min_close - entry_close) / entry_close) * 100

            results.append({
                'date': date,
                'year': daily_subset.iloc[0]['datetime'].year,
                'near_range': near_range,
                'movement_pct': movement_pct,
                'entry_close': entry_close,
                'last_5min_close': last_5min_close,
                'strategy': strategy
            })

# Process higher and lower trending days with stop-loss
for date in trending_higher_dates:
    prev_date = get_previous_trading_day(date, unique_dates)
    if prev_date:
        process_day_with_stoploss(date, prev_date, 'Higher High POC', min, near_ranges)

for date in trending_lower_dates:
    prev_date = get_previous_trading_day(date, unique_dates)
    if prev_date:
        process_day_with_stoploss(date, prev_date, 'Lower Low POC', max, near_ranges)

# Create DataFrame from results
results_df = pd.DataFrame(results)

# Calculate total percentage change including stop-loss hits
total_pct_change = results_df.groupby('near_range')['movement_pct'].sum()

# Print or use total_pct_change as needed
print("TOTAL PERCENTAGE CHANGE (Including stop-loss hits):")
print(total_pct_change)

yearly_profit_percentage = results_df.groupby('year')['movement_pct'].sum()

print("YEARLY PROFIT PERCENTAGE:")
print(yearly_profit_percentage)

bins = [-float('inf'), -2, -1, 0, 1, 1.5, 2, float('inf')]
labels = ['<-2%', '-2%--1%', '-1%-0%', '0%-1%', '1%-1.5%', '1.5%-2%', '>2%']
results_df['movement_range'] = pd.cut(results_df['movement_pct'], bins=bins, labels=labels)
range_counts = results_df.groupby(['near_range', 'movement_range']).size().unstack(fill_value=0)

print(" ")
print("TOTAL PERCENTAGE CHANGE:")
print(total_pct_change)
print(" ")
print("MOVEMENT PERCENTAGE RANGE COUNTS:")
print(range_counts)


TOTAL PERCENTAGE CHANGE (Including stop-loss hits):
near_range
0.0125    190.996992
Name: movement_pct, dtype: float64
YEARLY PROFIT PERCENTAGE:
year
2018    14.662285
2019    29.371066
2020    82.947313
2021    35.312894
2022    28.703433
Name: movement_pct, dtype: float64
 
TOTAL PERCENTAGE CHANGE:
near_range
0.0125    190.996992
Name: movement_pct, dtype: float64
 
MOVEMENT PERCENTAGE RANGE COUNTS:
movement_range  <-2%  -2%--1%  -1%-0%  0%-1%  1%-1.5%  1.5%-2%  >2%
near_range                                                         
0.0125             0       10     298    453       35       17   21


In [25]:
results_df

,date,year,near_range,movement_pct,entry_close,last_5min_close,strategy,movement_range
0,2018-01-04,2018,0.015,-0.078607,25570.10,25550.00,Higher High POC,-1%-0%
1,2018-01-08,2018,0.015,0.096661,25760.10,25785.00,Higher High POC,0%-1%
2,2018-01-09,2018,0.015,-0.049661,25775.00,25762.20,Higher High POC,-1%-0%
3,2018-01-15,2018,0.015,0.135893,26050.00,26085.40,Higher High POC,0%-1%
4,2018-01-16,2018,0.015,-0.054168,26030.10,26016.00,Higher High POC,-1%-0%
...,...,...,...,...,...,...,...,...
872,2022-12-08,2022,0.015,0.030778,44025.25,44011.70,Lower Low POC,0%-1%
873,2022-12-16,2022,0.015,0.327574,43700.00,43556.85,Lower Low POC,0%-1%
874,2022-12-19,2022,0.015,0.123116,43780.00,43726.10,Lower Low POC,0%-1%
875,2022-12-21,2022,0.015,-0.019784,42964.75,42973.25,Lower Low POC,-1%-0%


# Local Maxi and Local Min

In [5]:
df = pd.DataFrame(data_)
df['datetime'] = pd.to_datetime(df['datetime'])

# Extract date
df['date'] = df['datetime'].dt.date

# Group by date and aggregate unique local_max and local_min
result = df.groupby('date').agg({
    'local_max': lambda x: list(x.unique()),
    'local_min': lambda x: list(x.unique())
}).reset_index()

result

,date,local_max,local_min
0,2018-01-01,[25675.7],[25481.3]
1,2018-01-02,[25554.6],[25360.0]
2,2018-01-03,[25550.0],[25432.8]
3,2018-01-04,[25573.3],[25436.3]
4,2018-01-05,[25720.0],[25565.0]
...,...,...,...
1125,2022-12-26,[43037.95],[41869.75]
1126,2022-12-27,[43126.95],[42615.0]
1127,2022-12-28,[43228.65],[42900.75]
1128,2022-12-29,[43452.55],[42688.4]


In [10]:
### CORRECT DONE
import pandas as pd

data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])

# Prepare daily local_max and local_min
df = data.copy()
df['date'] = df['datetime'].dt.date

# Group by date and aggregate unique local_max and local_min
result = df.groupby('date').agg({
    'local_max': lambda x: list(x.unique()),
    'local_min': lambda x: list(x.unique())
}).reset_index()

# Create a dictionary for easy lookup
local_max_min_dict = result.set_index('date').to_dict(orient='index')

# Adding previous day's local_max and local_min to the data
data['prev_local_max'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('local_max', [None])[0])
data['prev_local_min'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('local_min', [None])[0])

class Trade:
    def __init__(self, entry_price, exit_price, trade_type, entry_date, exit_date, local_crossed):
        self.entry_price = entry_price
        self.exit_price = exit_price
        self.trade_type = trade_type
        self.entry_date = entry_date
        self.exit_date = exit_date
        self.local_crossed = local_crossed

def combined_strategy(data):
    trades = []
    in_trade = False
    total_profit_percentage = 0
    daily_profit_percentage = []

    current_trade_date = None

    for i in range(1, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        if not in_trade and (current_trade_date is None or candle['datetime'].date() != current_trade_date):
            # Short Sell Strategy
            if candle['close'] >= candle['prev_local_max'] and candle['close'] > prev_candle['vah']:
                entry_price = candle['close']
                trade_type = "short"
                entry_date = candle['datetime']
                stop_loss_price = entry_price * 1.015  # Stop loss for short trade
                in_trade = True
                current_trade_date = candle['datetime'].date()
                local_crossed = candle['prev_local_max']

            # Long Buy Strategy
            elif candle['close'] <= candle['prev_local_min'] and candle['close'] < prev_candle['val']:
                entry_price = candle['close']
                trade_type = "long"
                entry_date = candle['datetime']
                stop_loss_price = entry_price * 0.985  # Stop loss for long trade
                in_trade = True
                current_trade_date = candle['datetime'].date()
                local_crossed = candle['prev_local_min']

        while in_trade and i < len(data):
            next_candle = data.iloc[i]
            # Check for stop loss
            if trade_type == "short" and next_candle['high'] >= stop_loss_price:
                exit_price = stop_loss_price  # Exit at stop loss price
                profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False
            elif trade_type == "long" and next_candle['low'] <= stop_loss_price:
                exit_price = stop_loss_price  # Exit at stop loss price
                profit_percentage = ((exit_price - entry_price) / entry_price) * 100
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False

            # Force end-of-day exit at 15:15:00
            if in_trade and next_candle['datetime'].time() == pd.Timestamp("15:15:00").time():
                if trade_type == "short":
                    exit_price = next_candle['close']
                    profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                elif trade_type == "long":
                    exit_price = next_candle['close']
                    profit_percentage = ((exit_price - entry_price) / entry_price) * 100

                daily_profit_percentage.append({'Date': candle['datetime'].date(), 'Daily Profit Percentage': profit_percentage})
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False
                break

            i += 1

    if trades:
        total_profit_percentage = sum(
            [(trade.entry_price - trade.exit_price) if trade.trade_type == "short" else (trade.exit_price - trade.entry_price) for trade in trades]
        ) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = combined_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Ensure 'Date' column is datetime type
daily_profit['Date'] = pd.to_datetime(daily_profit['Date'])

# Group daily profits by year
daily_profit['Year'] = daily_profit['Date'].dt.year
yearly_profit_percentage = daily_profit.groupby('Year')['Daily Profit Percentage'].sum()

# Print the yearly profit percentages
print("\nYearly total profits (Percentage):")
for year, profit_percentage in yearly_profit_percentage.items():
    print(f"{year}: {profit_percentage:.2f}%")

# Calculate absolute yearly profits based on entry price
yearly_absolute_profits = {}
initial_entry_price = trades[0].entry_price
for year, group in daily_profit.groupby('Year'):
    absolute_profit = initial_entry_price * group['Daily Profit Percentage'].sum() / 100
    yearly_absolute_profits[year] = abs(absolute_profit)

# Print the yearly absolute profits
print("\nYearly total profits (Absolute Value):")
for year, profit in yearly_absolute_profits.items():
    print(f"{year}: {profit:.2f}")

# Group daily profits by year and month
daily_profit['Month'] = daily_profit['Date'].dt.month
monthly_profit_percentage = daily_profit.groupby(['Year', 'Month'])['Daily Profit Percentage'].sum()

# Print the monthly profit percentages
print("\nMonthly total profits (Percentage):")
for (year, month), profit_percentage in monthly_profit_percentage.items():
    print(f"{year}-{month:02d}: {profit_percentage:.2f}%")

# Create a DataFrame for trades with the required columns
trade_data = {
    'ENTRY_TYPE': [trade.trade_type for trade in trades],
    'ENTRY': [trade.entry_price for trade in trades],
    'ENTRY_TIME': [trade.entry_date for trade in trades],
    'EXIT': [trade.exit_price for trade in trades],
    'EXIT_TIME': [trade.exit_date for trade in trades],
    'LOCAL_CROSSED': [trade.local_crossed for trade in trades],
    'PROFIT_OR_LOSS_PERCENTAGE': [
        ((trade.exit_price - trade.entry_price) / trade.entry_price * 100) if trade.trade_type == "long" else
        ((trade.entry_price - trade.exit_price) / trade.entry_price * 100)
        for trade in trades
    ]
}

trads = pd.DataFrame(trade_data)
trads.to_csv('local2.csv', index=True)
trads


Total profit percentage: 168.71543916206141
Total trades taken: 636
Daily profit percentage:
            Date  Daily Profit Percentage
0    2018-01-02                 0.175948
1    2018-01-04                -0.450135
2    2018-01-05                -0.317138
3    2018-01-09                 0.233443
4    2018-01-10                 0.094716
..          ...                      ...
579  2022-12-23                -1.031316
580  2022-12-27                 0.000000
581  2022-12-28                 0.462848
582  2022-12-29                 1.333570
583  2022-12-30                 0.579513

[584 rows x 2 columns]

Yearly total profits (Percentage):
2018: 25.60%
2019: 37.01%
2020: 66.20%
2021: 31.44%
2022: 51.89%

Yearly total profits (Absolute Value):
2018: 6503.47
2019: 9401.30
2020: 16817.55
2021: 7988.11
2022: 13182.62

Monthly total profits (Percentage):
2018-01: 2.01%
2018-02: 2.48%
2018-03: 3.09%
2018-04: 3.30%
2018-05: -1.47%
2018-06: 4.73%
2018-07: 0.79%
2018-08: -1.22%
2018-09: 2.07%
201

,ENTRY_TYPE,ENTRY,ENTRY_TIME,EXIT,EXIT_TIME,LOCAL_CROSSED,PROFIT_OR_LOSS_PERCENTAGE
0,long,25405.20,2018-01-02 10:00:00,25449.90,2018-01-02 15:15:00,25481.30,0.175948
1,short,25570.10,2018-01-04 15:25:00,25685.20,2018-01-05 15:15:00,25550.00,-0.450135
2,short,25604.00,2018-01-05 09:15:00,25685.20,2018-01-05 15:15:00,25573.30,-0.317138
3,long,25702.20,2018-01-09 13:20:00,25762.20,2018-01-09 15:15:00,25708.00,0.233443
4,long,25655.70,2018-01-10 11:00:00,25680.00,2018-01-10 15:15:00,25692.30,0.094716
...,...,...,...,...,...,...,...
631,long,42377.90,2022-12-23 09:15:00,41940.85,2022-12-23 15:15:00,42560.00,-1.031316
632,short,43118.00,2022-12-27 15:15:00,43118.00,2022-12-27 15:15:00,43037.95,0.000000
633,short,43199.95,2022-12-28 11:10:00,43000.00,2022-12-28 15:15:00,43126.95,0.462848
634,long,42843.65,2022-12-29 09:15:00,43415.00,2022-12-29 15:15:00,42900.75,1.333570


In [17]:
import pandas as pd

data = pd.DataFrame(data_)
data['datetime'] = pd.to_datetime(data['datetime'])

# Prepare daily local_max and local_min
df = data.copy()
df['date'] = df['datetime'].dt.date

# Group by date and aggregate unique local_max, local_min, vah, and val
result = df.groupby('date').agg({
    'local_max': lambda x: list(x.unique()),
    'local_min': lambda x: list(x.unique()),
    'vah': lambda x: list(x.unique()),
    'val': lambda x: list(x.unique())
}).reset_index()

# Create a dictionary for easy lookup
local_max_min_dict = result.set_index('date').to_dict(orient='index')

# Adding previous day's local_max, local_min, vah, and val to the data
data['prev_local_max'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('local_max', [None])[0])
data['prev_local_min'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('local_min', [None])[0])
data['prev_vah'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('vah', [None])[0])
data['prev_val'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('val', [None])[0])

class Trade:
    def __init__(self, entry_price, exit_price, trade_type, entry_date, exit_date, local_crossed):
        self.entry_price = entry_price
        self.exit_price = exit_price
        self.trade_type = trade_type
        self.entry_date = entry_date
        self.exit_date = exit_date
        self.local_crossed = local_crossed

def combined_strategy(data):
    trades = []
    in_trade = False
    total_profit_percentage = 0
    daily_profit_percentage = []

    current_trade_date = None

    for i in range(1, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        if not in_trade and (current_trade_date is None or candle['datetime'].date() != current_trade_date):
            # Short Sell Strategy
            if candle['close'] > prev_candle['prev_vah'] or candle['close'] >= candle['prev_local_max']:
                entry_price = candle['close']
                trade_type = "short"
                entry_date = candle['datetime']
                stop_loss_price = entry_price * 1.015  # Stop loss for short trade
                in_trade = True
                current_trade_date = candle['datetime'].date()
                local_crossed = candle['prev_local_max']

            # Long Buy Strategy
            elif candle['close'] < prev_candle['prev_val'] or candle['low'] <= candle['prev_local_min']:
                entry_price = candle['close']
                trade_type = "long"
                entry_date = candle['datetime']
                stop_loss_price = entry_price * 0.985  # Stop loss for long trade
                in_trade = True
                current_trade_date = candle['datetime'].date()
                local_crossed = candle['prev_local_min']

        while in_trade and i < len(data):
            next_candle = data.iloc[i]
            # Check for stop loss
            if trade_type == "short" and next_candle['high'] >= stop_loss_price:
                exit_price = stop_loss_price  # Exit at stop loss price
                profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False
            elif trade_type == "long" and next_candle['low'] <= stop_loss_price:
                exit_price = stop_loss_price  # Exit at stop loss price
                profit_percentage = ((exit_price - entry_price) / entry_price) * 100
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False

            # Force end-of-day exit at 15:15:00
            if in_trade and next_candle['datetime'].time() == pd.Timestamp("15:15:00").time():
                if trade_type == "short":
                    exit_price = next_candle['close']
                    profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                elif trade_type == "long":
                    exit_price = next_candle['close']
                    profit_percentage = ((exit_price - entry_price) / entry_price) * 100

                daily_profit_percentage.append({'Date': candle['datetime'].date(), 'Daily Profit Percentage': profit_percentage})
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False
                break

            i += 1

    if trades:
        total_profit_percentage = sum(
            [(trade.entry_price - trade.exit_price) if trade.trade_type == "short" else (trade.exit_price - trade.entry_price) for trade in trades]
        ) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = combined_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Ensure 'Date' column is datetime type
daily_profit['Date'] = pd.to_datetime(daily_profit['Date'])

# Group daily profits by year
daily_profit['Year'] = daily_profit['Date'].dt.year
yearly_profit_percentage = daily_profit.groupby('Year')['Daily Profit Percentage'].sum()

# Print the yearly profit percentages
print("\nYearly total profits (Percentage):")
for year, profit_percentage in yearly_profit_percentage.items():
    print(f"{year}: {profit_percentage:.2f}%")

# Calculate absolute yearly profits based on entry price
yearly_absolute_profits = {}
initial_entry_price = trades[0].entry_price
for year, group in daily_profit.groupby('Year'):
    absolute_profit = initial_entry_price * group['Daily Profit Percentage'].sum() / 100
    yearly_absolute_profits[year] = abs(absolute_profit)

# Print the yearly absolute profits
print("\nYearly total profits (Absolute Value):")
for year, profit in yearly_absolute_profits.items():
    print(f"{year}: {profit:.2f}")

# Group daily profits by year and month
daily_profit['Month'] = daily_profit['Date'].dt.month
monthly_profit_percentage = daily_profit.groupby(['Year', 'Month'])['Daily Profit Percentage'].sum()

# Print the monthly profit percentages
print("\nMonthly total profits (Percentage):")
for (year, month), profit_percentage in monthly_profit_percentage.items():
    print(f"{year}-{month:02d}: {profit_percentage:.2f}%")

# Create a DataFrame for trades with the required columns
trade_data = {
    'ENTRY_TYPE': [trade.trade_type for trade in trades],
    'ENTRY': [trade.entry_price for trade in trades],
    'ENTRY_TIME': [trade.entry_date for trade in trades],
    'EXIT': [trade.exit_price for trade in trades],
    'EXIT_TIME': [trade.exit_date for trade in trades],
    'LOCAL_CROSSED': [trade.local_crossed for trade in trades],
    'PROFIT_OR_LOSS_PERCENTAGE': [
        ((trade.exit_price - trade.entry_price) / trade.entry_price * 100) if trade.trade_type == "long" else
        ((trade.entry_price - trade.exit_price) / trade.entry_price * 100)
        for trade in trades
    ]
}

trades = pd.DataFrame(trade_data)
trades.to_csv('local3.csv', index=True)
trades


Total profit percentage: 10.088684189863244
Total trades taken: 1040
Daily profit percentage:
            Date  Daily Profit Percentage
0    2018-01-02                -0.082839
1    2018-01-03                -0.314924
2    2018-01-04                 0.415023
3    2018-01-05                -0.317138
4    2018-01-08                 0.017837
..          ...                      ...
870  2022-12-26                 2.051835
871  2022-12-27                -0.089370
872  2022-12-28                 0.113708
873  2022-12-29                 1.333570
874  2022-12-30                 0.579513

[875 rows x 2 columns]

Yearly total profits (Percentage):
2018: 22.58%
2019: 48.84%
2020: 90.88%
2021: 53.68%
2022: 35.18%

Yearly total profits (Absolute Value):
2018: 5750.38
2019: 12440.37
2020: 23148.17
2021: 13672.22
2022: 8961.26

Monthly total profits (Percentage):
2018-01: -0.35%
2018-02: 3.76%
2018-03: 4.82%
2018-04: 1.89%
2018-05: -1.67%
2018-06: 5.86%
2018-07: -0.96%
2018-08: -0.29%
2018-09: -2.97

,ENTRY_TYPE,ENTRY,ENTRY_TIME,EXIT,EXIT_TIME,LOCAL_CROSSED,PROFIT_OR_LOSS_PERCENTAGE
0,long,25471.00,2018-01-02 09:20:00,25449.9,2018-01-02 15:15:00,25481.30,-0.082839
1,long,25530.00,2018-01-03 09:15:00,25449.6,2018-01-03 15:15:00,25360.00,-0.314924
2,long,25444.40,2018-01-04 09:20:00,25550.0,2018-01-04 15:15:00,25432.80,0.415023
3,short,25604.00,2018-01-05 09:15:00,25685.2,2018-01-05 15:15:00,25573.30,-0.317138
4,short,25789.60,2018-01-08 09:15:00,25785.0,2018-01-08 15:15:00,NaN,0.017837
...,...,...,...,...,...,...,...
1035,long,41950.25,2022-12-26 09:15:00,42811.0,2022-12-26 15:15:00,NaN,2.051835
1036,short,43079.50,2022-12-27 09:20:00,43118.0,2022-12-27 15:15:00,43037.95,-0.089370
1037,short,43048.95,2022-12-28 09:15:00,43000.0,2022-12-28 15:15:00,43126.95,0.113708
1038,long,42843.65,2022-12-29 09:15:00,43415.0,2022-12-29 15:15:00,42900.75,1.333570


In [16]:
import pandas as pd

# Assuming 'data_' contains your trading data in a list format

data = pd.DataFrame(data_)
data['datetime'] = pd.to_datetime(data['datetime'])

# Prepare daily local_max and local_min
df = data.copy()
df['date'] = df['datetime'].dt.date

# Group by date and aggregate unique local_max, local_min, vah, and val
result = df.groupby('date').agg({
    'local_max': lambda x: list(x.unique()),
    'local_min': lambda x: list(x.unique()),
    'vah': lambda x: list(x.unique()),
    'val': lambda x: list(x.unique())
}).reset_index()

# Create a dictionary for easy lookup
local_max_min_dict = result.set_index('date').to_dict(orient='index')

# Adding previous day's local_max, local_min, vah, and val to the data
data['prev_local_max'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('local_max', [None])[0])
data['prev_local_min'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('local_min', [None])[0])
data['prev_vah'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('vah', [None])[0])
data['prev_val'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('val', [None])[0])

class Trade:
    def __init__(self, entry_price, exit_price, trade_type, entry_date, exit_date, local_crossed):
        self.entry_price = entry_price
        self.exit_price = exit_price
        self.trade_type = trade_type
        self.entry_date = entry_date
        self.exit_date = exit_date
        self.local_crossed = local_crossed

def combined_strategy(data):
    trades = []
    in_trade = False
    total_profit_percentage = 0
    daily_profit_percentage = []

    current_trade_date = None

    for i in range(1, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        if not in_trade and (current_trade_date is None or candle['datetime'].date() != current_trade_date):
            # Short Sell Strategy
            if candle['close'] > prev_candle['prev_vah'] or candle['close'] >= candle['prev_local_max']:
                entry_price = candle['close']
                trade_type = "short"
                entry_date = candle['datetime']
                stop_loss_price = entry_price * 1.015  # Stop loss for short trade
                in_trade = True
                current_trade_date = candle['datetime'].date()
                local_crossed = candle['prev_local_max']

            # Long Buy Strategy
            elif candle['close'] < prev_candle['prev_val'] or candle['low'] <= candle['prev_local_min']:
                entry_price = candle['close']
                trade_type = "long"
                entry_date = candle['datetime']
                stop_loss_price = entry_price * 0.985  # Stop loss for long trade
                in_trade = True
                current_trade_date = candle['datetime'].date()
                local_crossed = candle['prev_local_min']

        while in_trade and i < len(data):
            next_candle = data.iloc[i]
            # Check for stop loss
            if trade_type == "short" and next_candle['high'] >= stop_loss_price:
                exit_price = stop_loss_price  # Exit at stop loss price
                profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False
            elif trade_type == "long" and next_candle['low'] <= stop_loss_price:
                exit_price = stop_loss_price  # Exit at stop loss price
                profit_percentage = ((exit_price - entry_price) / entry_price) * 100
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False

            # Force end-of-day exit at 15:15:00
            if in_trade and next_candle['datetime'].time() == pd.Timestamp("15:15:00").time():
                if trade_type == "short":
                    exit_price = next_candle['close']
                    profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                elif trade_type == "long":
                    exit_price = next_candle['close']
                    profit_percentage = ((exit_price - entry_price) / entry_price) * 100

                daily_profit_percentage.append({'Date': candle['datetime'].date(), 'Daily Profit Percentage': profit_percentage})
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False
                break

            i += 1

    if trades:
        total_profit_percentage = sum(
            [((trade.entry_price - trade.exit_price) if trade.trade_type == "short" else (trade.exit_price - trade.entry_price)) for trade in trades]
        ) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = combined_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Aggregate trades by year to calculate yearly profit
yearly_trades = {}
for trade in trades:
    year = trade.entry_date.year
    if year not in yearly_trades:
        yearly_trades[year] = []
    yearly_trades[year].append(trade)

# Calculate yearly profit percentages
yearly_profit_percentage = {}
for year, trades_list in yearly_trades.items():
    yearly_profit_percentage[year] = sum(
        [(trade.exit_price - trade.entry_price) if trade.trade_type == "long" else (trade.entry_price - trade.exit_price) for trade in trades_list]
    ) / trades_list[0].entry_price * 100

# Print the yearly profit percentages
print("\nYearly profit percentages:")
for year, profit_percentage in yearly_profit_percentage.items():
    print(f"{year}: {profit_percentage:.2f}%")

# Aggregate trades by year and month to calculate monthly profit
monthly_trades = {}
for trade in trades:
    year = trade.entry_date.year
    month = trade.entry_date.month
    if (year, month) not in monthly_trades:
        monthly_trades[(year, month)] = []
    monthly_trades[(year, month)].append(trade)

# Calculate monthly profit percentages
monthly_profit_percentage = {}
for (year, month), trades_list in monthly_trades.items():
    monthly_profit_percentage[(year, month)] = sum(
        [(trade.exit_price - trade.entry_price) if trade.trade_type == "long" else (trade.entry_price - trade.exit_price) for trade in trades_list]
    ) / trades_list[0].entry_price * 100

# Print the monthly profit percentages
print("\nMonthly profit percentages:")
for (year, month), profit_percentage in monthly_profit_percentage.items():
    print(f"{year}-{month:02d}: {profit_percentage:.2f}%")

# Create a DataFrame for trades with the required columns
trade_data = {
    'ENTRY_TYPE': [trade.trade_type for trade in trades],
    'ENTRY': [trade.entry_price for trade in trades],
    'ENTRY_TIME': [trade.entry_date for trade in trades],
    'EXIT': [trade.exit_price for trade in trades],
    'EXIT_TIME': [trade.exit_date for trade in trades],
    'LOCAL_CROSSED': [trade.local_crossed for trade in trades],
    'PROFIT_OR_LOSS_PERCENTAGE': [
        ((trade.exit_price - trade.entry_price) / trade.entry_price * 100) if trade.trade_type == "long" else
        ((trade.entry_price - trade.exit_price) / trade.entry_price * 100)
        for trade in trades
    ]
}

trades_df = pd.DataFrame(trade_data)
#trades_df.to_csv('local3.csv', index=True)
print(trades_df)


Total profit percentage: 10.088684189863244
Total trades taken: 1040
Daily profit percentage:
            Date  Daily Profit Percentage
0    2018-01-02                -0.082839
1    2018-01-03                -0.314924
2    2018-01-04                 0.415023
3    2018-01-05                -0.317138
4    2018-01-08                 0.017837
..          ...                      ...
870  2022-12-26                 2.051835
871  2022-12-27                -0.089370
872  2022-12-28                 0.113708
873  2022-12-29                 1.333570
874  2022-12-30                 0.579513

[875 rows x 2 columns]

Yearly profit percentages:
2018: 5.45%
2019: 24.67%
2020: -29.97%
2021: 9.43%
2022: 3.43%

Monthly profit percentages:
2018-01: -1.95%
2018-02: 0.71%
2018-03: 1.75%
2018-04: 0.40%
2018-05: -1.75%
2018-06: 5.82%
2018-07: -1.03%
2018-08: -0.30%
2018-09: -4.41%
2018-10: 6.26%
2018-12: 0.22%
2019-01: 0.14%
2019-02: 4.09%
2019-03: -9.72%
2019-04: 6.96%
2019-05: -0.47%
2019-06: 2.57%
2019-07

In [4]:
## volume included
import pandas as pd

# Assuming 'data_' contains your trading data in a list format

data = pd.DataFrame(data_)
data['datetime'] = pd.to_datetime(data['datetime'])

# Prepare daily local_max and local_min
df = data.copy()
df['date'] = df['datetime'].dt.date

# Group by date and aggregate unique local_max, local_min, vah, and val
result = df.groupby('date').agg({
    'local_max': lambda x: list(x.unique()),
    'local_min': lambda x: list(x.unique()),
    'vah': lambda x: list(x.unique()),
    'val': lambda x: list(x.unique())
}).reset_index()

# Create a dictionary for easy lookup
local_max_min_dict = result.set_index('date').to_dict(orient='index')

# Adding previous day's local_max, local_min, vah, and val to the data
data['prev_local_max'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('local_max', [None])[0])
data['prev_local_min'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('local_min', [None])[0])
data['prev_vah'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('vah', [None])[0])
data['prev_val'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('val', [None])[0])

# Calculate the high volume threshold (e.g., 75th percentile)
high_volume_threshold = data['volume'].quantile(0.75)

class Trade:
    def __init__(self, entry_price, exit_price, trade_type, entry_date, exit_date, local_crossed):
        self.entry_price = entry_price
        self.exit_price = exit_price
        self.trade_type = trade_type
        self.entry_date = entry_date
        self.exit_date = exit_date
        self.local_crossed = local_crossed

def combined_strategy(data):
    trades = []
    in_trade = False
    total_profit_percentage = 0
    daily_profit_percentage = []

    current_trade_date = None

    for i in range(1, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        if not in_trade and (current_trade_date is None or candle['datetime'].date() != current_trade_date):
            # Short Sell Strategy
            if candle['close'] > prev_candle['prev_vah'] or candle['close'] >= candle['prev_local_max']:
                if candle['volume'] > high_volume_threshold:
                    trade_type = "long"  # Switch to buy if volume is high
                else:
                    trade_type = "short"
                entry_price = candle['close']
                entry_date = candle['datetime']
                stop_loss_price = entry_price * (1.015 if trade_type == "short" else 0.985)  # Stop loss for short or long trade
                in_trade = True
                current_trade_date = candle['datetime'].date()
                local_crossed = candle['prev_local_max']

            # Long Buy Strategy
            elif candle['close'] < prev_candle['prev_val'] or candle['low'] <= candle['prev_local_min']:
                if candle['volume'] > high_volume_threshold:
                    trade_type = "short"  # Switch to short sell if volume is high
                else:
                    trade_type = "long"
                entry_price = candle['close']
                entry_date = candle['datetime']
                stop_loss_price = entry_price * (1.015 if trade_type == "short" else 0.985)  # Stop loss for short or long trade
                in_trade = True
                current_trade_date = candle['datetime'].date()
                local_crossed = candle['prev_local_min']

        while in_trade and i < len(data):
            next_candle = data.iloc[i]
            # Check for stop loss
            if trade_type == "short" and next_candle['high'] >= stop_loss_price:
                exit_price = stop_loss_price  # Exit at stop loss price
                profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False
            elif trade_type == "long" and next_candle['low'] <= stop_loss_price:
                exit_price = stop_loss_price  # Exit at stop loss price
                profit_percentage = ((exit_price - entry_price) / entry_price) * 100
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False

            # Force end-of-day exit at 15:15:00
            if in_trade and next_candle['datetime'].time() == pd.Timestamp("15:15:00").time():
                if trade_type == "short":
                    exit_price = next_candle['close']
                    profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                elif trade_type == "long":
                    exit_price = next_candle['close']
                    profit_percentage = ((exit_price - entry_price) / entry_price) * 100

                daily_profit_percentage.append({'Date': candle['datetime'].date(), 'Daily Profit Percentage': profit_percentage})
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False
                break

            i += 1

    if trades:
        total_profit_percentage = sum(
            [((trade.entry_price - trade.exit_price) if trade.trade_type == "short" else (trade.exit_price - trade.entry_price)) for trade in trades]
        ) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = combined_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Aggregate trades by year to calculate yearly profit
yearly_trades = {}
for trade in trades:
    year = trade.entry_date.year
    if year not in yearly_trades:
        yearly_trades[year] = []
    yearly_trades[year].append(trade)

# Calculate yearly profit percentages
yearly_profit_percentage = {}
for year, trades_list in yearly_trades.items():
    yearly_profit_percentage[year] = sum(
        [(trade.exit_price - trade.entry_price) if trade.trade_type == "long" else (trade.entry_price - trade.exit_price) for trade in trades_list]
    ) / trades_list[0].entry_price * 100

# Print the yearly profit percentages
print("\nYearly profit percentages:")
for year, profit_percentage in yearly_profit_percentage.items():
    print(f"{year}: {profit_percentage:.2f}%")

# Aggregate trades by year and month to calculate monthly profit
monthly_trades = {}
for trade in trades:
    year = trade.entry_date.year
    month = trade.entry_date.month
    if (year, month) not in monthly_trades:
        monthly_trades[(year, month)] = []
    monthly_trades[(year, month)].append(trade)

# Calculate monthly profit percentages
monthly_profit_percentage = {}
for (year, month), trades_list in monthly_trades.items():
    monthly_profit_percentage[(year, month)] = sum(
        [(trade.exit_price - trade.entry_price) if trade.trade_type == "long" else (trade.entry_price - trade.exit_price) for trade in trades_list]
    ) / trades_list[0].entry_price * 100

# Print the monthly profit percentages
print("\nMonthly profit percentages:")
for (year, month), profit_percentage in monthly_profit_percentage.items():
    print(f"{year}-{month:02d}: {profit_percentage:.2f}%")

# Create a DataFrame for trades with the required columns
trade_data = {
    'ENTRY_TYPE': [trade.trade_type for trade in trades],
    'ENTRY': [trade.entry_price for trade in trades],
    'ENTRY_TIME': [trade.entry_date for trade in trades],
    'EXIT': [trade.exit_price for trade in trades],
    'EXIT_TIME': [trade.exit_date for trade in trades],
    'LOCAL_CROSSED': [trade.local_crossed for trade in trades],
    'PROFIT_OR_LOSS_PERCENTAGE': [
        ((trade.exit_price - trade.entry_price) / trade.entry_price * 100) if trade.trade_type == "long" else
        ((trade.entry_price - trade.exit_price) / trade.entry_price * 100)
        for trade in trades
    ]
}

trades_df = pd.DataFrame(trade_data)
#trades_df.to_csv('local3.csv', index=True)
print(trades_df)


Total profit percentage: -109.08486710376346
Total trades taken: 1040
Daily profit percentage:
            Date  Daily Profit Percentage
0    2018-01-02                -0.082839
1    2018-01-03                -0.314924
2    2018-01-04                 0.415023
3    2018-01-05                -0.317138
4    2018-01-08                 0.017837
..          ...                      ...
820  2022-12-23                 1.031316
821  2022-12-27                 0.089370
822  2022-12-28                -0.113708
823  2022-12-29                -1.333570
824  2022-12-30                -0.579513

[825 rows x 2 columns]

Yearly profit percentages:
2018: 5.54%
2019: 12.44%
2020: -51.43%
2021: 11.78%
2022: -54.35%

Monthly profit percentages:
2018-01: 1.42%
2018-02: -2.58%
2018-03: 1.87%
2018-04: 0.28%
2018-05: 3.22%
2018-06: 6.27%
2018-07: -1.32%
2018-08: -1.92%
2018-09: -1.37%
2018-10: -1.49%
2018-12: 1.32%
2019-01: 4.07%
2019-02: -1.50%
2019-03: -4.21%
2019-04: 3.59%
2019-05: -1.94%
2019-06: -0.46%
2

In [5]:
import pandas as pd

# Assuming 'data_' contains your trading data in a list format

data = pd.DataFrame(data_)
data['datetime'] = pd.to_datetime(data['datetime'])

# Prepare daily local_max and local_min
df = data.copy()
df['date'] = df['datetime'].dt.date

# Group by date and aggregate unique local_max, local_min, vah, and val
result = df.groupby('date').agg({
    'local_max': lambda x: list(x.unique()),
    'local_min': lambda x: list(x.unique()),
    'vah': lambda x: list(x.unique()),
    'val': lambda x: list(x.unique())
}).reset_index()

# Create a dictionary for easy lookup
local_max_min_dict = result.set_index('date').to_dict(orient='index')

# Adding previous day's local_max, local_min, vah, and val to the data
data['prev_local_max'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('local_max', [None])[0])
data['prev_local_min'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('local_min', [None])[0])
data['prev_vah'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('vah', [None])[0])
data['prev_val'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('val', [None])[0])

# Calculate the high volume threshold (e.g., 75th percentile)
high_volume_threshold = data['volume'].quantile(0.75)

class Trade:
    def __init__(self, entry_price, exit_price, trade_type, entry_date, exit_date, local_crossed):
        self.entry_price = entry_price
        self.exit_price = exit_price
        self.trade_type = trade_type
        self.entry_date = entry_date
        self.exit_date = exit_date
        self.local_crossed = local_crossed

def combined_strategy(data):
    trades = []
    in_trade = False
    total_profit_percentage = 0
    daily_profit_percentage = []

    current_trade_date = None

    for i in range(1, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        if not in_trade and (current_trade_date is None or candle['datetime'].date() != current_trade_date):
            # Short Sell Strategy
            if candle['close'] > prev_candle['prev_vah'] or candle['close'] >= candle['prev_local_max']:
                if candle['volume'] > high_volume_threshold:
                    trade_type = "short"  # Switch to buy if volume is high
                else:
                    trade_type = "long"
                entry_price = candle['close']
                entry_date = candle['datetime']
                stop_loss_price = entry_price * (1.015 if trade_type == "long" else 0.985)  # Stop loss for short or long trade
                in_trade = True
                current_trade_date = candle['datetime'].date()
                local_crossed = candle['prev_local_max']

            # Long Buy Strategy
            elif candle['close'] < prev_candle['prev_val'] or candle['low'] <= candle['prev_local_min']:
                if candle['volume'] > high_volume_threshold:
                    trade_type = "long"  # Switch to short sell if volume is high
                else:
                    trade_type = "short"
                entry_price = candle['close']
                entry_date = candle['datetime']
                stop_loss_price = entry_price * (1.015 if trade_type == "long" else 0.985)  # Stop loss for short or long trade
                in_trade = True
                current_trade_date = candle['datetime'].date()
                local_crossed = candle['prev_local_min']

        while in_trade and i < len(data):
            next_candle = data.iloc[i]
            # Check for stop loss
            if trade_type == "long" and next_candle['high'] >= stop_loss_price:
                exit_price = stop_loss_price  # Exit at stop loss price
                profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False
            elif trade_type == "short" and next_candle['low'] <= stop_loss_price:
                exit_price = stop_loss_price  # Exit at stop loss price
                profit_percentage = ((exit_price - entry_price) / entry_price) * 100
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False

            # Force end-of-day exit at 15:15:00
            if in_trade and next_candle['datetime'].time() == pd.Timestamp("15:15:00").time():
                if trade_type == "long":
                    exit_price = next_candle['close']
                    profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                elif trade_type == "short":
                    exit_price = next_candle['close']
                    profit_percentage = ((exit_price - entry_price) / entry_price) * 100

                daily_profit_percentage.append({'Date': candle['datetime'].date(), 'Daily Profit Percentage': profit_percentage})
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime'], local_crossed))
                in_trade = False
                break

            i += 1

    if trades:
        total_profit_percentage = sum(
            [((trade.entry_price - trade.exit_price) if trade.trade_type == "short" else (trade.exit_price - trade.entry_price)) for trade in trades]
        ) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = combined_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Aggregate trades by year to calculate yearly profit
yearly_trades = {}
for trade in trades:
    year = trade.entry_date.year
    if year not in yearly_trades:
        yearly_trades[year] = []
    yearly_trades[year].append(trade)

# Calculate yearly profit percentages
yearly_profit_percentage = {}
for year, trades_list in yearly_trades.items():
    yearly_profit_percentage[year] = sum(
        [(trade.exit_price - trade.entry_price) if trade.trade_type == "long" else (trade.entry_price - trade.exit_price) for trade in trades_list]
    ) / trades_list[0].entry_price * 100

# Print the yearly profit percentages
print("\nYearly profit percentages:")
for year, profit_percentage in yearly_profit_percentage.items():
    print(f"{year}: {profit_percentage:.2f}%")

# Aggregate trades by year and month to calculate monthly profit
monthly_trades = {}
for trade in trades:
    year = trade.entry_date.year
    month = trade.entry_date.month
    if (year, month) not in monthly_trades:
        monthly_trades[(year, month)] = []
    monthly_trades[(year, month)].append(trade)

# Calculate monthly profit percentages
monthly_profit_percentage = {}
for (year, month), trades_list in monthly_trades.items():
    monthly_profit_percentage[(year, month)] = sum(
        [(trade.exit_price - trade.entry_price) if trade.trade_type == "long" else (trade.entry_price - trade.exit_price) for trade in trades_list]
    ) / trades_list[0].entry_price * 100

# Print the monthly profit percentages
print("\nMonthly profit percentages:")
for (year, month), profit_percentage in monthly_profit_percentage.items():
    print(f"{year}-{month:02d}: {profit_percentage:.2f}%")

# Create a DataFrame for trades with the required columns
trade_data = {
    'ENTRY_TYPE': [trade.trade_type for trade in trades],
    'ENTRY': [trade.entry_price for trade in trades],
    'ENTRY_TIME': [trade.entry_date for trade in trades],
    'EXIT': [trade.exit_price for trade in trades],
    'EXIT_TIME': [trade.exit_date for trade in trades],
    'LOCAL_CROSSED': [trade.local_crossed for trade in trades],
    'PROFIT_OR_LOSS_PERCENTAGE': [
        ((trade.exit_price - trade.entry_price) / trade.entry_price * 100) if trade.trade_type == "long" else
        ((trade.entry_price - trade.exit_price) / trade.entry_price * 100)
        for trade in trades
    ]
}

trades_df = pd.DataFrame(trade_data)
#trades_df.to_csv('local3.csv', index=True)
print(trades_df)


Total profit percentage: 109.08486710376346
Total trades taken: 1040
Daily profit percentage:
            Date  Daily Profit Percentage
0    2018-01-02                -0.082839
1    2018-01-03                -0.314924
2    2018-01-04                 0.415023
3    2018-01-05                -0.317138
4    2018-01-08                 0.017837
..          ...                      ...
820  2022-12-23                 1.031316
821  2022-12-27                 0.089370
822  2022-12-28                -0.113708
823  2022-12-29                -1.333570
824  2022-12-30                -0.579513

[825 rows x 2 columns]

Yearly profit percentages:
2018: -5.54%
2019: -12.44%
2020: 51.43%
2021: -11.78%
2022: 54.35%

Monthly profit percentages:
2018-01: -1.42%
2018-02: 2.58%
2018-03: -1.87%
2018-04: -0.28%
2018-05: -3.22%
2018-06: -6.27%
2018-07: 1.32%
2018-08: 1.92%
2018-09: 1.37%
2018-10: 1.49%
2018-12: -1.32%
2019-01: -4.07%
2019-02: 1.50%
2019-03: 4.21%
2019-04: -3.59%
2019-05: 1.94%
2019-06: 0.46%
20

In [6]:
trades_df.head(60)

,ENTRY_TYPE,ENTRY,ENTRY_TIME,EXIT,EXIT_TIME,LOCAL_CROSSED,PROFIT_OR_LOSS_PERCENTAGE
0,short,25471.0,2018-01-02 09:20:00,25449.9000,2018-01-02 15:15:00,25481.3,0.082839
1,short,25530.0,2018-01-03 09:15:00,25449.6000,2018-01-03 15:15:00,25360.0,0.314924
2,short,25444.4,2018-01-04 09:20:00,25550.0000,2018-01-04 15:15:00,25432.8,-0.415023
3,long,25604.0,2018-01-05 09:15:00,25685.2000,2018-01-05 15:15:00,25573.3,0.317138
4,long,25789.6,2018-01-08 09:15:00,25785.0000,2018-01-08 15:15:00,NaN,-0.017837
5,short,25711.3,2018-01-09 09:20:00,25762.2000,2018-01-09 15:15:00,25708.0,-0.197967
6,short,25734.3,2018-01-10 09:15:00,25680.0000,2018-01-10 15:15:00,25692.3,0.211002
7,short,25634.3,2018-01-11 09:15:00,25724.9000,2018-01-11 15:15:00,25630.0,-0.353433
8,long,25779.2,2018-01-12 09:15:00,25788.6000,2018-01-12 15:15:00,25753.1,0.036464
9,long,25877.8,2018-01-15 09:15:00,26085.4000,2018-01-15 15:15:00,NaN,0.802232


In [10]:
import pandas as pd



data = pd.DataFrame(data_)
data['datetime'] = pd.to_datetime(data['datetime'])

# Prepare daily local_max and local_min
df = data.copy()
df['date'] = df['datetime'].dt.date

# Group by date and aggregate unique local_max, local_min, vah, and val
result = df.groupby('date').agg({
    'vah': lambda x: list(x.unique()),
    'val': lambda x: list(x.unique())
}).reset_index()

# Create a dictionary for easy lookup
local_max_min_dict = result.set_index('date').to_dict(orient='index')

# Adding previous day's vah and val to the data
data['prev_vah'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('vah', [None])[0])
data['prev_val'] = data['datetime'].dt.date.map(lambda x: local_max_min_dict.get(x - pd.Timedelta(days=1), {}).get('val', [None])[0])

# Calculate rolling volume of the last 3 candles
data['rolling_volume'] = data['volume'].rolling(window=3).sum()

class Trade:
    def __init__(self, entry_price, exit_price, trade_type, entry_date, exit_date):
        self.entry_price = entry_price
        self.exit_price = exit_price
        self.trade_type = trade_type
        self.entry_date = entry_date
        self.exit_date = exit_date

def combined_strategy(data):
    trades = []
    in_trade = False
    total_profit_percentage = 0
    daily_profit_percentage = []

    current_trade_date = None

    for i in range(2, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        if not in_trade and (current_trade_date is None or candle['datetime'].date() != current_trade_date):
            # Short Sell Strategy
            if candle['close'] > prev_candle['prev_vah'] and candle['volume'] > prev_candle['rolling_volume']:
                entry_price = candle['close']
                trade_type = "long"
                entry_date = candle['datetime']
                stop_loss_price = entry_price *  0.985   # Stop loss for short trade
                in_trade = True
                current_trade_date = candle['datetime'].date()

            # Long Buy Strategy
            elif candle['close'] < prev_candle['prev_val'] and candle['volume'] > prev_candle['rolling_volume']:
                entry_price = candle['close']
                trade_type = "short"
                entry_date = candle['datetime']
                stop_loss_price = entry_price * 1.015  # Stop loss for long trade
                in_trade = True
                current_trade_date = candle['datetime'].date()

        while in_trade and i < len(data):
            next_candle = data.iloc[i]
            # Check for stop loss
            if trade_type == "short" and next_candle['high'] >= stop_loss_price:
                exit_price = stop_loss_price  # Exit at stop loss price
                profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime']))
                in_trade = False
            elif trade_type == "long" and next_candle['low'] <= stop_loss_price:
                exit_price = stop_loss_price  # Exit at stop loss price
                profit_percentage = ((exit_price - entry_price) / entry_price) * 100
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime']))
                in_trade = False

            # Force end-of-day exit at 15:15:00
            if in_trade and next_candle['datetime'].time() == pd.Timestamp("15:15:00").time():
                if trade_type == "short":
                    exit_price = next_candle['close']
                    profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                elif trade_type == "long":
                    exit_price = next_candle['close']
                    profit_percentage = ((exit_price - entry_price) / entry_price) * 100

                daily_profit_percentage.append({'Date': candle['datetime'].date(), 'Daily Profit Percentage': profit_percentage})
                trades.append(Trade(entry_price, exit_price, trade_type, entry_date, next_candle['datetime']))
                in_trade = False
                break

            i += 1

    if trades:
        total_profit_percentage = sum(
            [(trade.entry_price - trade.exit_price) if trade.trade_type == "short" else (trade.exit_price - trade.entry_price) for trade in trades]
        ) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = combined_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Ensure 'Date' column is datetime type
daily_profit['Date'] = pd.to_datetime(daily_profit['Date'])

# Group daily profits by year
daily_profit['Year'] = daily_profit['Date'].dt.year
yearly_profit_percentage = daily_profit.groupby('Year')['Daily Profit Percentage'].sum()

# Print the yearly profit percentages
print("\nYearly total profits (Percentage):")
for year, profit_percentage in yearly_profit_percentage.items():
    print(f"{year}: {profit_percentage:.2f}%")

# Calculate absolute yearly profits based on entry price
yearly_absolute_profits = {}
initial_entry_price = trades[0].entry_price
for year, group in daily_profit.groupby('Year'):
    absolute_profit = initial_entry_price * group['Daily Profit Percentage'].sum() / 100
    yearly_absolute_profits[year] = abs(absolute_profit)

# Print the yearly absolute profits
print("\nYearly total profits (Absolute Value):")
for year, profit in yearly_absolute_profits.items():
    print(f"{year}: {profit:.2f}")

# Group daily profits by year and month
daily_profit['Month'] = daily_profit['Date'].dt.month
monthly_profit_percentage = daily_profit.groupby(['Year', 'Month'])['Daily Profit Percentage'].sum()

# Print the monthly profit percentages
print("\nMonthly total profits (Percentage):")
for (year, month), profit_percentage in monthly_profit_percentage.items():
    print(f"{year}-{month:02d}: {profit_percentage:.2f}%")

# Create a DataFrame for trades with the required columns
trade_data = {
    'ENTRY_TYPE': [trade.trade_type for trade in trades],
    'ENTRY': [trade.entry_price for trade in trades],
    'ENTRY_TIME': [trade.entry_date for trade in trades],
    'EXIT': [trade.exit_price for trade in trades],
    'EXIT_TIME': [trade.exit_date for trade in trades],
    'PROFIT_OR_LOSS_PERCENTAGE': [
        ((trade.exit_price - trade.entry_price) / trade.entry_price * 100) if trade.trade_type == "long" else
        ((trade.entry_price - trade.exit_price) / trade.entry_price * 100)
        for trade in trades
    ]
}

trades = pd.DataFrame(trade_data)
# trades.to_csv('local3.csv', index=True)
print(trades)


Total profit percentage: -108.17673765338508
Total trades taken: 811
Daily profit percentage:
            Date  Daily Profit Percentage
0    2018-01-02                -0.125896
1    2018-01-03                -0.274690
2    2018-01-04                -0.377940
3    2018-01-05                 0.241968
4    2018-01-09                -0.197967
..          ...                      ...
662  2022-12-22                 0.986665
663  2022-12-23                 1.031316
664  2022-12-28                -0.462848
665  2022-12-29                -1.202825
666  2022-12-30                -0.347897

[667 rows x 2 columns]

Yearly total profits (Percentage):
2018: 4.83%
2019: 17.49%
2020: 66.76%
2021: 25.32%
2022: 11.93%

Yearly total profits (Absolute Value):
2018: 1228.30
2019: 4445.45
2020: 16968.06
2021: 6436.41
2022: 3033.01

Monthly total profits (Percentage):
2018-01: -1.30%
2018-02: 1.47%
2018-03: -3.21%
2018-04: -1.36%
2018-05: 1.44%
2018-06: -0.90%
2018-07: -0.85%
2018-08: 2.09%
2018-09: 4.33%
2

# Merging

In [61]:
import glob
import csv
import os
import time

start = time.time()

path = "/content/gdrive/MyDrive/bank_nifty data/csv"
all_files = glob.glob(os.path.join(path, "*.csv"))
cols = ['time', 'symbol', 'open', 'high', 'low', 'close', 'volume', 'oi', 'expiry', 'strike', 'instrument_type', 'Date']

output_file = 'dataset.csv'
with open(output_file, 'w', newline='') as out:
    writer = csv.writer(out)
    writer.writerow(cols)

for file in all_files:
    date = os.path.basename(file).split('.')[0]
    year = date[-6:]
    if year == '012018':  # Using string comparison
        with open(file, 'r') as f, open(output_file, 'a', newline='') as out:
            reader = csv.reader(f)
            writer = csv.writer(out)
            next(reader)  # Skip the header row
            for row in reader:
                row.append(date)
                writer.writerow(row)

end = time.time()
print("Execution time:", end - start)


Execution time: 2.9923579692840576


In [62]:
import pandas as pd
data1=pd.read_csv("/content/dataset.csv")
print(data1.shape)
data1.tail()

(508110, 12)


,time,symbol,open,high,low,close,volume,oi,expiry,strike,instrument_type,Date
508105,15:01:00,BANKNIFTY28MAR1828000CE,400.0,400.0,395.0,395.0,200,6360,28-03-2018,28000,CE,30012018
508106,15:05:00,BANKNIFTY28MAR1828000CE,395.0,395.0,380.0,380.0,120,6600,28-03-2018,28000,CE,30012018
508107,15:06:00,BANKNIFTY28MAR1828000CE,380.0,390.0,380.0,390.0,80,6720,28-03-2018,28000,CE,30012018
508108,13:52:00,BANKNIFTY28MAR1828000PE,950.0,950.0,950.0,950.0,400,0,28-03-2018,28000,PE,30012018
508109,13:32:00,BANKNIFTY28MAR1829000CE,150.0,150.0,150.0,150.0,40,40,28-03-2018,29000,CE,30012018


In [63]:

import pandas as pd
def preprocess_date(date_string):
    date_str = str(date_string)
    if len(date_str) == 8:
        # If the date string is in the format 'DDMMYYYY', convert it to 'DD-MM-YYYY'
        return date_str[:2] + '-' + date_str[2:4] + '-' + date_str[4:]
    else:
        date_str = '0' + date_str
        return date_str[:2] + '-' + date_str[2:4] + '-' + date_str[4:]
data1['Preprocessed_Date'] = data1['Date'].apply(preprocess_date)

In [68]:
sorted_data = data1.sort_values(by=['Preprocessed_Date', 'instrument_type', 'strike', 'time']).reset_index(drop=True)
sorted_data

,time,symbol,open,high,low,close,volume,oi,expiry,strike,instrument_type,Date,Preprocessed_Date
0,13:32:00,BANKNIFTY25JAN1823200CE,2333.00,2333.00,2333.00,2333.00,40,100760,25-01-2018,23200,CE,1012018,01-01-2018
1,13:36:00,BANKNIFTY25JAN1823200CE,2333.00,2336.00,2333.00,2336.00,40,100800,25-01-2018,23200,CE,1012018,01-01-2018
2,14:52:00,BANKNIFTY25JAN1823200CE,2336.00,2336.00,2292.95,2292.95,40,100760,25-01-2018,23200,CE,1012018,01-01-2018
3,15:19:00,BANKNIFTY25JAN1823200CE,2292.95,2292.95,2215.00,2215.00,40,100760,25-01-2018,23200,CE,1012018,01-01-2018
4,09:27:00,BANKNIFTY25JAN1823500CE,2074.45,2074.45,2074.45,2074.45,40,25880,25-01-2018,23500,CE,1012018,01-01-2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
508105,14:42:00,BANKNIFTY22FEB1829000PE,1730.15,1730.15,1527.70,1527.70,40,6160,22-02-2018,29000,PE,31012018,31-01-2018
508106,14:44:00,BANKNIFTY22FEB1829000PE,1527.70,1527.70,1500.30,1500.30,40,6160,22-02-2018,29000,PE,31012018,31-01-2018
508107,14:49:00,BANKNIFTY22FEB1829000PE,1500.30,1500.30,1488.60,1488.60,40,6160,22-02-2018,29000,PE,31012018,31-01-2018
508108,15:21:00,BANKNIFTY22FEB1829000PE,1488.60,1548.70,1488.60,1548.70,80,6160,22-02-2018,29000,PE,31012018,31-01-2018


In [70]:
sorted_data = data1.sort_values(by=['Preprocessed_Date', 'instrument_type', 'strike', 'time']).reset_index(drop=True)


import pandas as pd
from datetime import timedelta

sorted_data['Preprocessed_Date'] = pd.to_datetime(sorted_data['Preprocessed_Date'], format='%d-%m-%Y')
sorted_data['expiry'] = pd.to_datetime(sorted_data['expiry'], format='%d-%m-%Y')

# Function to find nearest expiry date within 10 days tolerance
def nearest_expiry(group):
    min_expiry = group['expiry'].min()
    min_date = group['Preprocessed_Date'].min()
    if (min_expiry - min_date).days <= 10:
        return min_expiry
    else:
        return pd.NaT  # Return NaT if no expiry date within 10 days

# Group by Preprocessed_Date and find the nearest expiry date
nearest_expiries = sorted_data.groupby('Preprocessed_Date').apply(nearest_expiry).reset_index(name='nearest_expiry')

print(nearest_expiries)


#sorted_data.head(60)

   Preprocessed_Date nearest_expiry
0         2018-01-01     2018-01-04
1         2018-01-02     2018-01-04
2         2018-01-03     2018-01-04
3         2018-01-04     2018-01-04
4         2018-01-05     2018-01-11
5         2018-01-08     2018-01-11
6         2018-01-09     2018-01-11
7         2018-01-10     2018-01-11
8         2018-01-11     2018-01-11
9         2018-01-12     2018-01-18
10        2018-01-15     2018-01-18
11        2018-01-16     2018-01-18
12        2018-01-17     2018-01-18
13        2018-01-18     2018-01-18
14        2018-01-19     2018-01-25
15        2018-01-22     2018-01-25
16        2018-01-23     2018-01-25
17        2018-01-24     2018-01-25
18        2018-01-25     2018-01-25
19        2018-01-29     2018-02-01
20        2018-01-30     2018-02-01
21        2018-01-31     2018-02-01


In [71]:
sorted_data['Preprocessed_Date'] = pd.to_datetime(sorted_data['Preprocessed_Date'], format='%d-%m-%Y')
sorted_data['expiry'] = pd.to_datetime(sorted_data['expiry'], format='%d-%m-%Y')

# Function to find the nearest expiry date within 10 days tolerance
def nearest_expiry(group):
    min_expiry = group['expiry'].min()
    min_date = group['Preprocessed_Date'].min()
    if (min_expiry - min_date).days <= 10:
        return min_expiry
    else:
        return pd.NaT  # Return NaT if no expiry date within 10 days

# Group by Preprocessed_Date and find the nearest expiry date
nearest_expiries = sorted_data.groupby('Preprocessed_Date').apply(nearest_expiry).reset_index(name='nearest_expiry')

# Merge with the original DataFrame to retain only the rows with the combinations of Preprocessed_Date and nearest expiry
filtered_data = pd.merge(sorted_data, nearest_expiries, how='inner', left_on=['Preprocessed_Date', 'expiry'], right_on=['Preprocessed_Date', 'nearest_expiry'])

# Drop the helper column 'nearest_expiry' if needed
filtered_data.drop(columns=['nearest_expiry'], inplace=True)
filtered_data


,time,symbol,open,high,low,close,volume,oi,expiry,strike,instrument_type,Date,Preprocessed_Date
0,14:14:00,BANKNIFTY04JAN1824500CE,1019.9,1019.9,1019.90,1019.90,40,1200,2018-01-04,24500,CE,1012018,2018-01-01
1,14:15:00,BANKNIFTY04JAN1824500CE,1019.9,1025.0,1019.90,1025.00,40,1240,2018-01-04,24500,CE,1012018,2018-01-01
2,14:24:00,BANKNIFTY04JAN1824500CE,1025.0,1025.0,1022.00,1022.00,40,1280,2018-01-04,24500,CE,1012018,2018-01-01
3,14:58:00,BANKNIFTY04JAN1824500CE,1022.0,1022.0,949.55,949.55,40,1320,2018-01-04,24500,CE,1012018,2018-01-01
4,15:14:00,BANKNIFTY04JAN1824500CE,888.4,889.0,887.30,888.20,200,1320,2018-01-04,24500,CE,1012018,2018-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263238,15:03:00,BANKNIFTY01FEB1828300PE,855.0,855.0,855.00,855.00,40,80,2018-02-01,28300,PE,31012018,2018-01-31
263239,13:18:00,BANKNIFTY01FEB1828500PE,1239.1,1240.7,1239.10,1240.70,80,80,2018-02-01,28500,PE,31012018,2018-01-31
263240,13:37:00,BANKNIFTY01FEB1828500PE,1234.0,1240.7,1234.00,1234.00,80,0,2018-02-01,28500,PE,31012018,2018-01-31
263241,13:54:00,BANKNIFTY01FEB1828500PE,1234.0,1258.0,1234.00,1258.00,40,80,2018-02-01,28500,PE,31012018,2018-01-31


In [98]:
import pandas as pd
import numpy as np

data = filtered_data.copy()  # Make a copy to avoid modifying original data
data['time'] = pd.to_datetime(data['time'])

time_range = pd.date_range(start='09:15:00', end='15:30:00', freq='5T')
expanded_data = pd.DataFrame()

grouped_data = data.groupby(['Preprocessed_Date', 'instrument_type'])

for (Preprocessed_Date, instrument_type), subset in grouped_data:
    # Group the subset by strike and resample each group to 5-minute intervals
    resampled_subset = subset.set_index('time').groupby(['strike', pd.Grouper(freq='5T', level='time'), 'expiry']).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum',
        'oi': 'last'
    }).reset_index(level=['strike', 'time', 'expiry'])

    # Forward filling
    resampled_subset[['high', 'low']] = resampled_subset[['high', 'low']].ffill()

    # Interpolating
    resampled_subset[['volume', 'oi']] = resampled_subset[['volume', 'oi']].replace(0, np.nan)
    resampled_subset[['volume', 'oi']] = resampled_subset[['volume', 'oi']].interpolate(method='linear')
    resampled_subset[['volume', 'oi']] = resampled_subset[['volume', 'oi']].fillna(0).round(2)
    resampled_subset[['open', 'close']] = resampled_subset[['open', 'close']].interpolate(method='linear').round(2)

    # Reset index to include 'strike', 'time', and 'expiry' as columns
    resampled_subset.reset_index(inplace=True)

    # Set 'symbol' and 'expiry' columns to match original subset values
    for col in ['symbol', 'expiry']:
        resampled_subset[col] = subset[col].iloc[0]

    # Assign Preprocessed_Date and instrument_type to each row
    resampled_subset['Preprocessed_Date'] = Preprocessed_Date
    resampled_subset['instrument_type'] = instrument_type

    # Concatenate resampled_subset to expanded_data
    expanded_data = pd.concat([expanded_data, resampled_subset], ignore_index=True)

# Sort and reset index of expanded_data
expanded_data.sort_values(by=['Preprocessed_Date', 'instrument_type', 'strike', 'time'], inplace=True)
expanded_data.reset_index(drop=True, inplace=True)

# Convert 'time' column to time format
expanded_data['time'] = pd.to_datetime(expanded_data['time']).dt.time

expanded_data


<ipython-input-98-b701cfc17487>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['time'] = pd.to_datetime(data['time'])


,index,strike,time,expiry,open,high,low,close,volume,oi,symbol,Preprocessed_Date,instrument_type
0,0,24500,14:10:00,2018-01-04,1019.9,1019.9,1019.90,1019.90,40,1200.0,BANKNIFTY04JAN1824500CE,2018-01-01,CE
1,1,24500,14:15:00,2018-01-04,1019.9,1025.0,1019.90,1025.00,40,1240.0,BANKNIFTY04JAN1824500CE,2018-01-01,CE
2,2,24500,14:20:00,2018-01-04,1025.0,1025.0,1022.00,1022.00,40,1280.0,BANKNIFTY04JAN1824500CE,2018-01-01,CE
3,3,24500,14:55:00,2018-01-04,1022.0,1022.0,949.55,949.55,40,1320.0,BANKNIFTY04JAN1824500CE,2018-01-01,CE
4,4,24500,15:10:00,2018-01-04,888.4,889.0,887.30,888.20,200,1320.0,BANKNIFTY04JAN1824500CE,2018-01-01,CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66200,1948,28300,15:00:00,2018-02-01,855.0,855.0,855.00,855.00,40,80.0,BANKNIFTY01FEB1823300PE,2018-01-31,PE
66201,1949,28500,13:15:00,2018-02-01,1239.1,1240.7,1239.10,1240.70,80,80.0,BANKNIFTY01FEB1823300PE,2018-01-31,PE
66202,1950,28500,13:35:00,2018-02-01,1234.0,1240.7,1234.00,1234.00,80,80.0,BANKNIFTY01FEB1823300PE,2018-01-31,PE
66203,1951,28500,13:50:00,2018-02-01,1234.0,1258.0,1234.00,1258.00,40,80.0,BANKNIFTY01FEB1823300PE,2018-01-31,PE


In [76]:

import pandas as pd
import os
from datetime import datetime

directory_path = '/content/gdrive/MyDrive/Fut_data'
data_list = []

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        try:
            date_str = filename.split('.')[0]  # Assuming the filename is like 'DDMMYYYY.csv'
            date_obj = datetime.strptime(date_str, '%d%m%Y')
            formatted_date = date_obj.strftime('%d-%m-%Y')
        except ValueError:
            print(f'{filename}: Filename does not match expected date format')
            continue
        if date_obj.year==2018:                                                                                   #change here!!!!!!!!!!!!!!!!!!!!!!!!
        #if date_obj.year==2018 and date_obj.month==12:
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path)

            # Considering ATM value from the first row
            if 'open' in df.columns:
                atm_value = df['open'].iloc[0]
                # Appending the formatted date and the ATM value to the data list
                data_list.append((formatted_date, atm_value))
            else:
                print(f'{formatted_date}: "open" column not found')

data_list.sort(key=lambda x: datetime.strptime(x[0], '%d-%m-%Y'))
for formatted_date, atm_value in data_list:
    print(f'{formatted_date}: ATM = {atm_value}')

01-01-2018: ATM = 25570.0
02-01-2018: ATM = 25490.0
03-01-2018: ATM = 25440.0
04-01-2018: ATM = 25394.75
05-01-2018: ATM = 25534.0
08-01-2018: ATM = 25699.85
09-01-2018: ATM = 25750.0
10-01-2018: ATM = 25696.2
11-01-2018: ATM = 25602.4
12-01-2018: ATM = 25710.0
15-01-2018: ATM = 25774.75
16-01-2018: ATM = 26121.0
17-01-2018: ATM = 25989.95
18-01-2018: ATM = 26610.25
19-01-2018: ATM = 26551.05
22-01-2018: ATM = 26889.9
23-01-2018: ATM = 27044.4
24-01-2018: ATM = 27310.05
25-01-2018: ATM = 27474.65
29-01-2018: ATM = 27382.0
30-01-2018: ATM = 27451.05
31-01-2018: ATM = 27222.0
01-02-2018: ATM = 27430.35
02-02-2018: ATM = 26994.95
05-02-2018: ATM = 26094.95
06-02-2018: ATM = 25201.1
07-02-2018: ATM = 26111.35
08-02-2018: ATM = 25741.0
09-02-2018: ATM = 25510.15
12-02-2018: ATM = 25602.5
14-02-2018: ATM = 25760.0
15-02-2018: ATM = 25325.0
16-02-2018: ATM = 25475.1
19-02-2018: ATM = 25300.05
20-02-2018: ATM = 25061.1
21-02-2018: ATM = 24980.0
22-02-2018: ATM = 24785.0
23-02-2018: ATM = 25002

In [99]:
import pandas as pd
import os
from datetime import datetime

# Directory path
directory_path = '/content/gdrive/MyDrive/Fut_data'
data_list = []

# Extract ATM values from CSV files
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        try:
            date_str = filename.split('.')[0]  # Assuming the filename is like 'DDMMYYYY.csv'
            date_obj = datetime.strptime(date_str, '%d%m%Y')
            formatted_date = date_obj.strftime('%d-%m-%Y')
        except ValueError:
            print(f'{filename}: Filename does not match expected date format')
            continue
        if date_obj.year == 2018:  # Filter by year
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path)
            # Considering ATM value from the first row
            if 'open' in df.columns:
                atm_value = df['open'].iloc[0]
                data_list.append((formatted_date, atm_value))
            else:
                print(f'{formatted_date}: "open" column not found')

# Sort the list by date
data_list.sort(key=lambda x: datetime.strptime(x[0], '%d-%m-%Y'))

# Create a dictionary from the sorted list
atm_dict = dict(data_list)



# Ensure Preprocessed_Date is in the same string format as used in atm_dict
expanded_data['Preprocessed_Date'] = expanded_data['Preprocessed_Date'].dt.strftime('%d-%m-%Y')

# Function to map preprocessed date to ATM value
def get_atm(date):
    return atm_dict.get(date, None)

# Adding a new column 'ATM' using the mapping function
expanded_data['atm'] = expanded_data['Preprocessed_Date'].apply(get_atm)

# Print the expanded_data to verify the 'atm' column
expanded_data


,index,strike,time,expiry,open,high,low,close,volume,oi,symbol,Preprocessed_Date,instrument_type,atm
0,0,24500,14:10:00,2018-01-04,1019.9,1019.9,1019.90,1019.90,40,1200.0,BANKNIFTY04JAN1824500CE,01-01-2018,CE,25570.0
1,1,24500,14:15:00,2018-01-04,1019.9,1025.0,1019.90,1025.00,40,1240.0,BANKNIFTY04JAN1824500CE,01-01-2018,CE,25570.0
2,2,24500,14:20:00,2018-01-04,1025.0,1025.0,1022.00,1022.00,40,1280.0,BANKNIFTY04JAN1824500CE,01-01-2018,CE,25570.0
3,3,24500,14:55:00,2018-01-04,1022.0,1022.0,949.55,949.55,40,1320.0,BANKNIFTY04JAN1824500CE,01-01-2018,CE,25570.0
4,4,24500,15:10:00,2018-01-04,888.4,889.0,887.30,888.20,200,1320.0,BANKNIFTY04JAN1824500CE,01-01-2018,CE,25570.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66200,1948,28300,15:00:00,2018-02-01,855.0,855.0,855.00,855.00,40,80.0,BANKNIFTY01FEB1823300PE,31-01-2018,PE,27222.0
66201,1949,28500,13:15:00,2018-02-01,1239.1,1240.7,1239.10,1240.70,80,80.0,BANKNIFTY01FEB1823300PE,31-01-2018,PE,27222.0
66202,1950,28500,13:35:00,2018-02-01,1234.0,1240.7,1234.00,1234.00,80,80.0,BANKNIFTY01FEB1823300PE,31-01-2018,PE,27222.0
66203,1951,28500,13:50:00,2018-02-01,1234.0,1258.0,1234.00,1258.00,40,80.0,BANKNIFTY01FEB1823300PE,31-01-2018,PE,27222.0


In [104]:
df = pd.DataFrame(expanded_data)


# Convert 'expiry' and 'Preprocessed_Date' to datetime
df['expiry'] = pd.to_datetime(df['expiry'], format='%d-%m-%Y')
df['Preprocessed_Date'] = pd.to_datetime(df['Preprocessed_Date'], format='%d-%m-%Y')

# Function to find the minimum expiry date for each Preprocessed_Date group and filter the group
def filter_min_expiry(group):
    min_expiry = group['expiry'].min()
    return group[group['expiry'] == min_expiry]

# Group by 'Preprocessed_Date' and apply the function
results = df.groupby('Preprocessed_Date').apply(filter_min_expiry).reset_index(drop=True)

results

,index,strike,time,expiry,open,high,low,close,volume,oi,symbol,Preprocessed_Date,instrument_type,atm
0,0,24500,14:10:00,2018-01-04,1019.9,1019.9,1019.90,1019.90,40,1200.0,BANKNIFTY04JAN1824500CE,2018-01-01,CE,25570.0
1,1,24500,14:15:00,2018-01-04,1019.9,1025.0,1019.90,1025.00,40,1240.0,BANKNIFTY04JAN1824500CE,2018-01-01,CE,25570.0
2,2,24500,14:20:00,2018-01-04,1025.0,1025.0,1022.00,1022.00,40,1280.0,BANKNIFTY04JAN1824500CE,2018-01-01,CE,25570.0
3,3,24500,14:55:00,2018-01-04,1022.0,1022.0,949.55,949.55,40,1320.0,BANKNIFTY04JAN1824500CE,2018-01-01,CE,25570.0
4,4,24500,15:10:00,2018-01-04,888.4,889.0,887.30,888.20,200,1320.0,BANKNIFTY04JAN1824500CE,2018-01-01,CE,25570.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66200,1948,28300,15:00:00,2018-02-01,855.0,855.0,855.00,855.00,40,80.0,BANKNIFTY01FEB1823300PE,2018-01-31,PE,27222.0
66201,1949,28500,13:15:00,2018-02-01,1239.1,1240.7,1239.10,1240.70,80,80.0,BANKNIFTY01FEB1823300PE,2018-01-31,PE,27222.0
66202,1950,28500,13:35:00,2018-02-01,1234.0,1240.7,1234.00,1234.00,80,80.0,BANKNIFTY01FEB1823300PE,2018-01-31,PE,27222.0
66203,1951,28500,13:50:00,2018-02-01,1234.0,1258.0,1234.00,1258.00,40,80.0,BANKNIFTY01FEB1823300PE,2018-01-31,PE,27222.0


In [120]:
df = pd.DataFrame(results)

# Group by 'atm' and select the maximum 'strike' less than 'atm' for each group
max_strikes = df[df['strike'] < df['atm']].groupby('atm')['strike'].max().reset_index()

# Merge with original DataFrame to get rows with max strike and atm
result1 = pd.merge(max_strikes, df, on=['atm', 'strike'], how='inner')

result1

,atm,strike,index,time,expiry,open,high,low,close,volume,oi,symbol,Preprocessed_Date,instrument_type
0,25394.75,25300,240,09:15:00,2018-01-04,70.05,82.00,60.00,62.65,438960,388840.0,BANKNIFTY04JAN1824000CE,2018-01-04,CE
1,25394.75,25300,241,09:20:00,2018-01-04,62.35,66.10,56.00,60.25,323080,434280.0,BANKNIFTY04JAN1824000CE,2018-01-04,CE
2,25394.75,25300,242,09:25:00,2018-01-04,59.70,65.80,59.65,65.00,211800,492400.0,BANKNIFTY04JAN1824000CE,2018-01-04,CE
3,25394.75,25300,243,09:30:00,2018-01-04,64.90,64.95,54.45,64.35,253840,550440.0,BANKNIFTY04JAN1824000CE,2018-01-04,CE
4,25394.75,25300,244,09:35:00,2018-01-04,65.00,65.00,58.00,58.75,186520,584880.0,BANKNIFTY04JAN1824000CE,2018-01-04,CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3305,27474.65,27400,1938,15:05:00,2018-01-25,32.00,33.00,2.50,2.60,877640,570720.0,BANKNIFTY25JAN1823200PE,2018-01-25,PE
3306,27474.65,27400,1939,15:10:00,2018-01-25,2.65,4.60,1.05,1.60,1174760,774840.0,BANKNIFTY25JAN1823200PE,2018-01-25,PE
3307,27474.65,27400,1940,15:15:00,2018-01-25,1.55,3.00,0.55,0.90,796040,888520.0,BANKNIFTY25JAN1823200PE,2018-01-25,PE
3308,27474.65,27400,1941,15:20:00,2018-01-25,0.90,0.95,0.10,0.15,644280,911360.0,BANKNIFTY25JAN1823200PE,2018-01-25,PE


In [121]:
result1['Preprocessed_Date'] = result1['Preprocessed_Date'].astype(str)

# Convert 'time' to string format that can be concatenated
result1['time'] = result1['time'].apply(lambda x: x.strftime('%H:%M:%S'))

# Combine 'Preprocessed_Date' and 'time' columns
result1['datetime'] = pd.to_datetime(result1['Preprocessed_Date'] + ' ' + result1['time'])

# Drop the original 'time' and 'Preprocessed_Date' columns if needed
result1 = result1.drop(columns=['time', 'Preprocessed_Date','symbol','index'])

# Sort by the new datetime_combined column if needed
result1 = result1.sort_values(by='datetime')
result1

,atm,strike,expiry,open,high,low,close,volume,oi,instrument_type,datetime
603,25570.0,25500,2018-01-04,130.00,136.85,115.00,136.50,84960,336800.0,CE,2018-01-01 09:15:00
679,25570.0,25500,2018-01-04,128.60,135.05,108.75,109.55,133520,344920.0,PE,2018-01-01 09:15:00
604,25570.0,25500,2018-01-04,137.25,159.70,135.95,146.75,105000,351320.0,CE,2018-01-01 09:20:00
680,25570.0,25500,2018-01-04,110.50,111.00,95.40,104.15,125080,392520.0,PE,2018-01-01 09:20:00
605,25570.0,25500,2018-01-04,146.80,149.60,143.05,143.15,44080,351560.0,CE,2018-01-01 09:25:00
...,...,...,...,...,...,...,...,...,...,...,...
2708,27222.0,27200,2018-02-01,142.90,151.95,142.75,145.75,42440,139960.0,PE,2018-01-31 15:20:00
2633,27222.0,27200,2018-02-01,343.70,352.90,336.05,338.55,12400,64760.0,CE,2018-01-31 15:20:00
2634,27222.0,27200,2018-02-01,338.55,351.35,336.25,336.25,16400,60280.0,CE,2018-01-31 15:25:00
2709,27222.0,27200,2018-02-01,145.75,145.75,134.30,135.00,46160,132480.0,PE,2018-01-31 15:25:00


In [129]:
result1 = pd.DataFrame(result1)

# Pivot to get oi data for CE and PE separately
oi_data = result1.pivot(index='datetime', columns='instrument_type', values='oi').reset_index()
oi_data.columns.name = None  # Remove columns name from pivot

# Rename columns for clarity
oi_data.rename(columns={'CE': 'ce_oi', 'PE': 'pe_oi'}, inplace=True)

# Print the resulting dataframe
print(oi_data)

                datetime     ce_oi     pe_oi
0    2018-01-01 09:15:00  336800.0  344920.0
1    2018-01-01 09:20:00  351320.0  392520.0
2    2018-01-01 09:25:00  351560.0  401400.0
3    2018-01-01 09:30:00  353480.0  412960.0
4    2018-01-01 09:35:00  353880.0  416320.0
...                  ...       ...       ...
1657 2018-01-31 15:10:00   70560.0  146960.0
1658 2018-01-31 15:15:00   68280.0  142440.0
1659 2018-01-31 15:20:00   64760.0  139960.0
1660 2018-01-31 15:25:00   60280.0  132480.0
1661 2018-01-31 15:30:00       NaN  132480.0

[1662 rows x 3 columns]


In [139]:
# Assuming oi_data and data_ are pandas DataFrames
# Merge oi_data columns into data_
data_['datetime'] = pd.to_datetime(data_['datetime'])

data0 = data_.merge(oi_data[['datetime', 'ce_oi', 'pe_oi']], on='datetime', how='left')
data0

,datetime,atm,open,high,low,close,volume,oi,max_pain_strike,intraday_pcr,...,gap_type,gap_filled,3_day_vah,5_day_vah,3_day_val,5_day_val,3_day_poc,5_day_poc,ce_oi,pe_oi
0,2018-01-01 09:15:00,25570.00,25634.00,25643.10,25608.00,25643.10,480.0,0.0,25700,0.8287,...,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,336800.0,344920.0
1,2018-01-01 09:20:00,25570.00,25631.50,25675.70,25631.50,25665.30,600.0,0.0,25600,0.7786,...,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,351320.0,392520.0
2,2018-01-01 09:25:00,25570.00,25665.30,25665.30,25655.10,25655.10,240.0,0.0,25700,0.7420,...,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,351560.0,401400.0
3,2018-01-01 09:30:00,25570.00,25647.70,25647.70,25647.70,25647.70,240.0,0.0,25600,0.7683,...,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,353480.0,412960.0
4,2018-01-01 09:35:00,25570.00,25647.70,25647.70,25647.00,25647.00,440.0,0.0,25700,0.7973,...,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,353880.0,416320.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92694,2022-12-30 15:10:00,43501.05,43320.05,43349.70,43302.45,43322.00,2650.0,168225.0,42500,0.6826,...,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7,NaN,NaN
92695,2022-12-30 15:15:00,43501.05,43317.30,43331.95,43290.00,43310.00,4950.0,168625.0,42000,0.6204,...,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7,NaN,NaN
92696,2022-12-30 15:20:00,43501.05,43307.35,43443.95,43293.25,43432.90,6600.0,168750.0,42500,0.5152,...,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7,NaN,NaN
92697,2022-12-30 15:25:00,43501.05,43433.60,43443.70,43377.05,43403.40,6100.0,168500.0,42000,0.4360,...,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7,NaN,NaN


In [140]:
result1.rename(columns={
    'strike': 'option_strike',
    'expiry': 'option_expiry',
    'open': 'option_open',
    'high': 'option_high',
    'low': 'option_low',
    'close': 'option_close',
    'volume': 'option_volume'
}, inplace=True)

# Merge renamed result1 columns into data0 on 'datetime'
data0 = pd.merge(data0, result1[['datetime', 'option_strike', 'option_expiry', 'option_open', 'option_high', 'option_low', 'option_close', 'option_volume']],
                 on='datetime', how='left')
data0

,datetime,atm,open,high,low,close,volume,oi,max_pain_strike,intraday_pcr,...,5_day_poc,ce_oi,pe_oi,option_strike,option_expiry,option_open,option_high,option_low,option_close,option_volume
0,2018-01-01 09:15:00,25570.00,25634.00,25643.10,25608.00,25643.10,480.0,0.0,25700,0.8287,...,NaN,336800.0,344920.0,25500.0,2018-01-04,130.00,136.85,115.00,136.50,84960.0
1,2018-01-01 09:15:00,25570.00,25634.00,25643.10,25608.00,25643.10,480.0,0.0,25700,0.8287,...,NaN,336800.0,344920.0,25500.0,2018-01-04,128.60,135.05,108.75,109.55,133520.0
2,2018-01-01 09:20:00,25570.00,25631.50,25675.70,25631.50,25665.30,600.0,0.0,25600,0.7786,...,NaN,351320.0,392520.0,25500.0,2018-01-04,137.25,159.70,135.95,146.75,105000.0
3,2018-01-01 09:20:00,25570.00,25631.50,25675.70,25631.50,25665.30,600.0,0.0,25600,0.7786,...,NaN,351320.0,392520.0,25500.0,2018-01-04,110.50,111.00,95.40,104.15,125080.0
4,2018-01-01 09:25:00,25570.00,25665.30,25665.30,25655.10,25655.10,240.0,0.0,25700,0.7420,...,NaN,351560.0,401400.0,25500.0,2018-01-04,146.80,149.60,143.05,143.15,44080.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94342,2022-12-30 15:10:00,43501.05,43320.05,43349.70,43302.45,43322.00,2650.0,168225.0,42500,0.6826,...,43244.7,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
94343,2022-12-30 15:15:00,43501.05,43317.30,43331.95,43290.00,43310.00,4950.0,168625.0,42000,0.6204,...,43244.7,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
94344,2022-12-30 15:20:00,43501.05,43307.35,43443.95,43293.25,43432.90,6600.0,168750.0,42500,0.5152,...,43244.7,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
94345,2022-12-30 15:25:00,43501.05,43433.60,43443.70,43377.05,43403.40,6100.0,168500.0,42000,0.4360,...,43244.7,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [141]:
data0.columns

Index(['datetime', 'atm', 'open', 'high', 'low', 'close', 'volume', 'oi',
       'max_pain_strike', 'intraday_pcr', 'cumulative_pcr',
       'most_active_strikes', 'vwap', 'awap', 'macd', 'local_max', 'local_min',
       'vah', 'val', 'poc', 'value_area_width(%)', 'highest_above_vah',
       'lowest_below_val', 'gap_type', 'gap_filled', '3_day_vah', '5_day_vah',
       '3_day_val', '5_day_val', '3_day_poc', '5_day_poc', 'ce_oi', 'pe_oi',
       'option_strike', 'option_expiry', 'option_open', 'option_high',
       'option_low', 'option_close', 'option_volume'],
      dtype='object')

In [143]:
save_path = '/content/gdrive/MyDrive/With_oi_data/data0.csv'
data0.to_csv(save_path, index=True)
print('Saved')

Saved
